In [ ]:
# Đây là template ví dụ template:
data_exmp = [
    {
        "template": "PacketResponder <*> for block <*> terminating",
        "message": "PacketResponder 1 for block blk_1073741825 terminating"
    },
    {
        "template": "BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to <*> size <*>",
        "message": "BLOCK* NameSystem.addStoredBlock: blockMap updated: 127.0.0.1:50010 is added to blockpool size 67108864"
    },
    {
        "template": "Received block <*> of size <*> from <*>",
        "message": "Received block blk_1073741827 of size 1048576 from 127.0.0.1"
    },
    {
        "template": "Receiving block <*> src: <*>:<*> dest: <*>:<*>",
        "message": "Receiving block blk_1073741827 src: 127.0.0.1:50010 dest: 127.0.0.1:50020"
    },
    {
        "template": "BLOCK* NameSystem.allocateBlock: <*> <*>",
        "message": "BLOCK* NameSystem.allocateBlock: user1 blk_1073741826"
    },
    {
        "template": "Verification succeeded for <*>",
        "message": "Verification succeeded for blk_1073741828"
    },
    {
        "template": "Deleting block <*> file <*>",
        "message": "Deleting block blk_1073741829 file /tmp/hadoop/blk_1073741829"
    },
    {
        "template": "<*>:<*> Served block <*> to <*>",
        "message": "127.0.0.1:50010 Served block blk_1073741830 to client1"
    },
    {
        "template": "<*>:<*>:Got exception while serving <*> to <*>:",
        "message": "127.0.0.1:50010:Got exception while serving blk_1073741831 to client2:"
    },
    {
        "template": "BLOCK* NameSystem.delete: <*> is added to invalidSet of <*>:<*>",
        "message": "BLOCK* NameSystem.delete: blk_1073741832 is added to invalidSet of 127.0.0.1:50010"
    },
    {
        "template": "<*>:<*> Starting thread to transfer block <*> to <*>:<*>",
        "message": "127.0.0.1:50010 Starting thread to transfer block blk_1073741833 to 127.0.0.1:50011"
    },
    {
        "template": "BLOCK* ask <*>:<*> to delete <*>",
        "message": "BLOCK* ask 127.0.0.1:50010 to delete blk_1073741834"
    },
    {
        "template": "Received block <*> src: <*>:<*> dest: <*>:<*> of size <*>",
        "message": "Received block blk_1073741835 src: 127.0.0.1:50010 dest: 127.0.0.1:50020 of size 1048576"
    },
    {
        "template": "BLOCK* ask <*>:<*> to replicate <*> to datanode(s) <*>:<*>",
        "message": "BLOCK* ask 127.0.0.1:50010 to replicate blk_1073741836 to datanode(s) 127.0.0.1:50012"
    }
]


In [ ]:
# File GA_calculator.py trong UNLEASH
import pandas as pd
from tqdm import tqdm

def evaluate(df_groundtruth, df_parsedlog, filter_templates=None):
    """ Đánh giá độ chính xác của việc phân tích log bằng cách so sánh dữ liệu ground truth và kết quả phân tích log. Phương thức này thực hiện các bước sau:
        1. Loại bỏ các dòng không hợp lệ trong dữ liệu ground truth (các dòng có giá trị NaN trong cột `EventTemplate`).
        2. Gọi hàm `get_accuracy` để tính toán các chỉ số độ chính xác:
            - GA (Grouping Accuracy): Độ chính xác nhóm.
            - FGA (F-Measure of Grouping Accuracy): Trung bình điều hòa giữa độ chính xác và độ bao phủ.
        3. In kết quả GA và FGA ra màn hình.
        4. Trả về giá trị GA và FGA.

    Args:
        df_groundtruth (pd.DataFrame): DataFrame chứa dữ liệu ground truth với cột `EventTemplate`.
        df_parsedlog (pd.DataFrame): DataFrame chứa kết quả phân tích log với cột `EventTemplate`.
        filter_templates (list, optional): Danh sách các template cần lọc. Nếu không được cung cấp, tất cả các template sẽ được sử dụng.

    Returns:
        tuple: (GA, FGA), trong đó:
            - GA (float): Độ chính xác nhóm (Grouping Accuracy).
            - FGA (float): F-Measure của độ chính xác nhóm.
    """ 
    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index       # Lấy các chỉ số (index) không null từ df_groundtruth 
    df_groundtruth = df_groundtruth.loc[null_logids]                                    # Lọc df_groundtruth để chỉ giữ lại các dòng không null 
    df_parsedlog = df_parsedlog.loc[null_logids]                                        # Tương tự
    GA, FGA = get_accuracy(df_groundtruth['EventTemplate'], df_parsedlog['EventTemplate'])
    print('Grouping_Accuracy (GA): %.4f, FGA: %.4f,'%(GA, FGA))
    return GA, FGA

def get_accuracy(series_groundtruth, series_parsedlog, filter_templates=None):
    """ Tính toán các chỉ số đánh giá độ chính xác giữa kết quả phân tích log và ground truth. Phương thức này tính toán hai chỉ số chính, gồm GA, FGA.

    Args:
        series_groundtruth (pandas.Series): Chuỗi chứa các template ground truth.
        series_parsedlog (pandas.Series): Chuỗi chứa các template từ kết quả phân tích log.
        filter_templates (list, optional): Danh sách các template cần lọc. Nếu không được cung cấp, tất cả các template sẽ được sử dụng để đánh giá độ chính xác. Nó được sử dụng để lọc các templates cụ thể trong quá trình tính toán độ chính xác. Nó cho phép người dùng chỉ tập trung vào một tập hợp con các template thay vì toàn bộ dữ liệu.

    Returns:
        tuple: (GA, FGA), trong đó:
            - GA (float): Độ chính xác nhóm (Grouping Accuracy).
            - FGA (float): F-Measure của độ chính xác nhóm.

    Example:
        >>> series_groundtruth = pd.Series(['A', 'B', 'A', 'C'])
        >>> series_parsedlog = pd.Series(['A', 'B', 'A', 'D'])
        >>> get_accuracy(series_groundtruth, series_parsedlog)
        (0.75, 0.6667)
    """
    
    series_groundtruth_valuecounts = series_groundtruth.value_counts()      # dtype: int64, A:3, B:2, C:1
    series_parsedlog_valuecounts = series_parsedlog.value_counts()          # Tương tự
    df_combined = pd.concat([series_groundtruth, series_parsedlog], axis=1, keys=['groundtruth', 'parsedlog'])
    grouped_df = df_combined.groupby('groundtruth')                         # Kết hợp và nhóm dữ liệu theo cột 'groundtruth'
    accurate_events = 0                                                     # Số lượng sự kiện chính xác          
    accurate_templates = 0                                                  # Số lượng template chính xác
    if filter_templates is not None:
        filter_identify_templates = set()                                   # Tập hợp lưu trữ các template đã được xác định
    
    for ground_truthId, group in tqdm(grouped_df):
        series_parsedlog_logId_valuecounts = group['parsedlog'].value_counts()          # Lấy số lượng các template phân tích được bởi công cụ cùng một nhóm với một template trong ground_truth 
        if filter_templates is not None and ground_truthId in filter_templates:         # Nếu template này đã có trong filter_templates và filter_templates không phải là None
            for parsed_eventId in series_parsedlog_logId_valuecounts.index:             # ==> parsed_eventId = "A" hoặc "B"
                filter_identify_templates.add(parsed_eventId)
        if series_parsedlog_logId_valuecounts.size == 1:
            parsed_eventId = series_parsedlog_logId_valuecounts.index[0]
            if len(group) == series_parsedlog[series_parsedlog == parsed_eventId].size: # Nếu có một template duy nhất trong nhóm và số lượng dòng trong nhóm bằng với số lượng dòng trong series_parsedlog ==> Nhóm log đúng
                if (filter_templates is None) or (ground_truthId in filter_templates):
                    accurate_events += len(group)
                    accurate_templates += 1
    if filter_templates is not None:
        GA = float(accurate_events) / len(series_groundtruth[series_groundtruth.isin(filter_templates)])        # Tính toán dựa trên số lượng trong filter_templates (phương thức isin() trả về True nếu giá trị trong series nằm trong filter_templates)
        PGA = float(accurate_templates) / len(filter_identify_templates)
        RGA = float(accurate_templates) / len(filter_templates)
    else:
        GA = float(accurate_events) / len(series_groundtruth)
        PGA = float(accurate_templates) / len(series_parsedlog_valuecounts)
        RGA = float(accurate_templates) / len(series_groundtruth_valuecounts)
    # print(FGA, RGA)
    FGA = 0.0
    if PGA != 0 or RGA != 0:
        FGA = 2 * (PGA * RGA) / (PGA + RGA)
    return GA, FGA

In [ ]:
# File PA_calculator.py trong UNLEASH

import pandas as pd
import regex as re


def post_process_tokens(tokens, punc):
    """ Phương thức thực hiện hậu xử lý danh sách các token cho trước, loại bỏ các ký tự không cần thiết và chuẩn hóa các token.
    Chức năng chính:
        1. Nếu một token chứa chuỗi "<*>", toàn bộ token sẽ được thay thế bằng "<*>".
        2. Với các token khác, loại bỏ các ký tự không thuộc danh sách `punc`, không phải khoảng trắng (' '), 
           hoặc không nằm trong danh sách ký tự đặc biệt `excluded_str` (gồm '=', '|', '(', ')').
           
    Args:
        tokens (list): Danh sách các token cần xử lý.
        punc (str): Chuỗi chứa các ký tự phân cách và ký tự không cần thiết.    
    
    Returns:
        list: Danh sách các token đã được xử lý.
        
    Examples:
        >>> tokens = ["hello", "world<*>", "test|case", "a(b)c"]
        >>> punc = "!\"#$%&'()+,-/:;=?@[\\]^_`{|}~"
        >>> post_process_tokens(tokens, punc)
        ['hello', '<*>', 'test|case', 'abc']
    """
    excluded_str = ['=', '|', '(', ')']                         # Các ký tự đặc biệt cần giữ lại
    for i in range(len(tokens)):
        if tokens[i].find("<*>") != -1:
            tokens[i] = "<*>"
        else:
            new_str = ""
            for s in tokens[i]:
                if (s not in punc and s != ' ') or s in excluded_str:
                    new_str += s
            tokens[i] = new_str
    return tokens

def message_split(message):
    """ Tách chuỗi đầu vào thành các token dựa trên các ký tự phân cách và thực hiện xử lý hậu kỳ. 
    Chức năng chính:
        1. Tách chuỗi dựa trên các ký tự phân cách (khoảng trắng, dấu câu, ...).
        2. Loại bỏ các token rỗng hoặc không hợp lệ.
        3. Chuẩn hóa các token bằng cách loại bỏ các ký tự không cần thiết.
        4. Loại bỏ các token `<*>` liên tiếp.

    Args:
        message (str): Chuỗi đầu vào cần tách.

    Returns:
        list: Danh sách các token đã được xử lý.

    Examples:
        >>> message = "Hello, world! This is a test <*> <*>."
        >>> message_split(message)
        ['Hello', ',', 'world', '!', 'This', 'is', 'a', 'test', '<*>', '.']
    """
    
    punc = "!\"#$%&'()+,-/:;=?@.[\]^_`{|}~"                     # Các ký tự được sử dụng để phân tách
    splitters = "\s\\" + "\\".join(punc)                        # Tạo biểu thức chính quy để tách chuỗi
    splitter_regex = re.compile("([{}]+)".format(splitters))    
    
    # Ex: message = "Hello, world! This is a test <*> <*>."
    tokens = re.split(splitter_regex, message)          # tokens = ['Hello', ',', 'world', '!', 'This', 'is', 'a', 'test', '<*>', '<*>', '.']
    tokens = list(filter(lambda x: x != "", tokens))    # tokens = ['Hello', ',', 'world', '!', 'This', 'is', 'a', 'test', '<*>', '<*>', '.']
    
    tokens = post_process_tokens(tokens, punc)          # tokens = ['Hello', 'world', 'This', 'is', 'a', 'test', '<*>', '<*>']
    tokens = [ 
        token.strip() 
        for token in tokens 
        if token != "" and token != ' ' 
    ] # Loại bỏ các token rỗng hoặc khoảng trắng
    tokens = [ 
        token 
        for idx, token in enumerate(tokens) 
        if not (token == "<*>" and idx > 0 and tokens[idx - 1] == "<*>")
    ] # Loại bỏ các token `<*>` liên tiếp
    return tokens


def calculate_similarity(template1, template2):
    """ Phương thức đo lường mức độ giống nhau giữa hai chuỗi văn bản (template1 và template2) bằng cách sử dụng Chỉ số Jaccard.
    Chỉ số Jaccard là tỷ lệ giữa số lượng phần tử chung của hai tập hợp và tổng số phần tử của cả hai tập hợp.
    
    Args:
        template1 (str): Chuỗi văn bản đầu tiên.
        template2 (str): Chuỗi văn bản thứ hai. 
        
    Returns:
        float: Chỉ số Jaccard giữa hai chuỗi văn bản.    
    """
    template1 = message_split(template1)
    template2 = message_split(template2)
    intersection = len(set(template1).intersection(set(template2)))             # Giao giữa hai template
    union = (len(template1) + len(template2)) - intersection                    # Hợp giữa hai template
    return intersection / union


def calculate_parsing_accuracy(groundtruth_df, parsedresult_df, filter_templates=None):
    """ Tính toán độ chính xác của quá trình phân tích cú pháp (Parsing Accuracy - PA). 
    Quy trình hoạt động:
        1. Nếu `filter_templates` được cung cấp, lọc dữ liệu thực tế và kết quả phân tích để chỉ giữ lại các template trong danh sách này.
        2. So sánh cột `EventTemplate` giữa hai DataFrame để đếm số lượng message được phân tích đúng.
        3. Tính toán độ chính xác phân tích cú pháp (PA) bằng cách chia số lượng message đúng cho tổng số message.
        4. In ra độ chính xác phân tích cú pháp với định dạng 4 chữ số thập phân.
    
    Args:
        groundtruth_df (pd.DataFrame): DataFrame chứa dữ liệu thực tế với cột `EventTemplate` và `Content`.
        parsedresult_df (pd.DataFrame): DataFrame chứa kết quả phân tích với cột `EventTemplate` và `Content`.
        filter_templates (list, optional): Danh sách các template cần sử dụng để tính toán.

    Returns:
        float: Độ chính xác của quá trình phân tích cú pháp (Parsing Accuracy - PA), được tính bằng tỷ lệ giữa số lượng message được phân tích đúng và tổng số message.

    Examples:
        >>> groundtruth_df = pd.DataFrame({
        ...     'EventTemplate': ['A', 'B', 'C'],
        ...     'Content': ['msg1', 'msg2', 'msg3']
        ... })
        >>> parsedresult_df = pd.DataFrame({
        ...     'EventTemplate': ['A', 'B', 'D'],
        ...     'Content': ['msg1', 'msg2', 'msg3']
        ... })
        >>> calculate_parsing_accuracy(groundtruth_df, parsedresult_df)
        Parsing_Accuracy (PA): 0.6667
        0.6667
    """
    if filter_templates is not None:            # Nếu có filter_templates, tính toán theo các template chỉ định
        groundtruth_df = groundtruth_df[groundtruth_df['EventTemplate'].isin(filter_templates)]
        parsedresult_df = parsedresult_df.loc[groundtruth_df.index]
        
    correctly_parsed_messages = parsedresult_df[['EventTemplate']].eq(groundtruth_df[['EventTemplate']]).values.sum()
    total_messages = len(parsedresult_df[['Content']])

    PA = float(correctly_parsed_messages) / total_messages

    print('Parsing_Accuracy (PA): {:.4f}'.format(PA))
    return PA

# Phương thức mới, tính toán độ tương đồng phân tích cú pháp:
def calculate_similarity_accuracy(groundtruth_df, parsedresult_df, filter_templates=None):
    """ Tính toán độ chính xác tương đồng giữa các template phân tích cú pháp và dữ liệu thực tế. Dùng được cho tất cả các trình phân tích cú pháp.
    
    Args:
        groundtruth_df (pd.DataFrame): DataFrame chứa dữ liệu thực tế với cột `EventTemplate` và `Content`.
        parsedresult_df (pd.DataFrame): DataFrame chứa kết quả phân tích với cột `EventTemplate` và `Content`.
        filter_templates (list, optional): Danh sách các template cần sử dụng để tính toán.

    Returns:
        float: Độ chính xác tương đồng giữa các template phân tích cú pháp và dữ liệu thực tế.

    Examples:
        >>> groundtruth_df = pd.DataFrame({
        ...     'EventTemplate': ['A', 'B', 'C'],
        ...     'Content': ['msg1', 'msg2', 'msg3']
        ... })
        >>> parsedresult_df = pd.DataFrame({
        ...     'EventTemplate': ['A', 'B', 'D'],
        ...     'Content': ['msg1', 'msg2', 'msg3']
        ... })
        >>> calculate_similarity_accuracy(groundtruth_df, parsedresult_df)
        Similarity_Accuracy (SA): 0.6667
        0.6667
    """
    if filter_templates is not None:
        groundtruth_df = groundtruth_df[groundtruth_df['EventTemplate'].isin(filter_templates)]
        parsedresult_df = parsedresult_df.loc[groundtruth_df.index]

    similarities = []
    for index in range(len(groundtruth_df)):
        similarities.append(calculate_similarity(groundtruth_df['EventTemplate'][index], parsedresult_df['EventTemplate'][index]))
    SA = sum(similarities) / len(similarities)
    print('Similarity_Accuracy (SA): {:.4f}'.format(SA))
    return SA

def calculate_parsing_accuracy_lstm(groundtruth_df, parsedresult_df, filter_templates=None):
    """ Tương tự, Tính toán độ chính xác của quá trình phân tích cú pháp (Parsing Accuracy - PA) cho các trình phân tích dựa trên ngữ nghĩa.
        
        Args:
            groundtruth_df (pd.DataFrame): DataFrame chứa dữ liệu thực tế với cột `EventTemplate` và `Content`.
            parsedresult_df (pd.DataFrame): DataFrame chứa kết quả phân tích với cột `EventTemplate` và `Content`.
            filter_templates (list, optional): Danh sách các template cần sử dụng để tính toán.

        Returns:
            float: Độ chính xác của quá trình phân tích cú pháp (Parsing Accuracy - PA), được tính bằng tỷ lệ giữa số lượng message được phân tích đúng và tổng số message.
        """
    if filter_templates is not None:
        groundtruth_df = groundtruth_df[groundtruth_df['EventTemplate'].isin(filter_templates)]
        parsedresult_df = parsedresult_df.loc[groundtruth_df.index]

    # Tương tự, nhưng thêm một phương thức tính toán (correct_lstm) để kiểm tra độ chính xác dành riêng cho các trình phân tích dựa trên ngữ nghĩa
    groundtruth_templates = list(groundtruth_df['EventTemplate'])
    parsedresult_templates = list(parsedresult_df['EventTemplate'])
    correctly_parsed_messages = 0
    for i in range(len(groundtruth_templates)):
        if correct_lstm(groundtruth_templates[i], parsedresult_templates[i]):
            correctly_parsed_messages += 1

    PA = float(correctly_parsed_messages) / len(groundtruth_templates)
    print('Parsing_Accuracy (PA): {:.4f}'.format(PA))
    return PA

def correct_lstm(groudtruth, parsedresult):
    """ Phương thức tính toán độ chính xác phân tích dành riêng cho các trình phân tích cú pháp dựa trên ngữ nghĩa. Bản chất, chỉ chỉnh sửa lại, lọc các nhiễu trong groudtruth để so sánh với parsedresult.

    Args:
        groudtruth (str): Chuỗi văn bản gốc (ground truth).
        parsedresult (str): Chuỗi văn bản đã được phân tích (parsed result).

    Returns:
        bool: Trả về True nếu hai danh sách từ giống nhau, ngược lại trả về False.
    """
    tokens1 = groudtruth.split(' ')
    tokens2 = parsedresult.split(' ')
    tokens1 = [
        "<*>" 
        if "<*>" in token else token 
        for token in tokens1
    ]       # Chỉnh sửa lại token trong groudtruth
    return tokens1 == tokens2


In [ ]:
# File common.py trong UNLEASH
from __future__ import print_function

import time
import regex as re
import os
import pandas as pd
import numpy as np
import argparse
from datetime import datetime
from natsort import natsorted


all_datasets = [
    "Proxifier",
    "Linux",
    "Apache",
    "Zookeeper",
    "Mac",
    "OpenStack",
    "HealthApp",
    "Hadoop",
    "HPC",
    "OpenSSH",
    "BGL",
    "HDFS",
    # "Android",
    "Spark",
    # "Windows",
    "Thunderbird",
]

datasets = ['HDFS', 'Hadoop', 'Spark', 'Zookeeper', 'OpenStack', 'BGL', 'HPC', 'Thunderbird', 'Windows', 'Linux', 'Mac', 'Android', 'HealthApp', 'Apache', 'OpenSSH', 'Proxifier']  


def sort_templates(templates):
    """ Sắp xếp danh sách các template theo độ dài giảm dần.
    Args:
        templates (list): Danh sách các template cần sắp xếp.
    Returns:
        list: Danh sách các template đã được sắp xếp theo thứ tự giảm dần của độ dài.
    """
    return sorted(templates, key=lambda x: len(x), reverse=True)


def get_pattern_from_template(template):
    """ Tạo một biểu thức chính quy (regular expression) từ một template. Template có thể chứa các ký tự đặc biệt và ký tự đại diện `<*>`. 
    Hàm này sẽ:
    - Thoát (Escape) các ký tự đặc biệt trong template để đảm bảo chúng không bị hiểu nhầm.
    - Thay thế các khoảng trắng bằng biểu thức chính quy `\s+` để khớp với một hoặc nhiều khoảng trắng.
    - Thay thế ký tự đại diện `<*>` bằng `(\S+?)` để khớp với một hoặc nhiều ký tự không phải khoảng trắng.    
    - Thêm dấu `^` và `$` để biểu thức chính quy khớp toàn bộ chuỗi.

    Args:
        template (str): template cần chuyển đổi thành biểu thức chính quy.

    Returns:
        str: Biểu thức chính quy được tạo từ template.
    
    Examples:
        >>> get_pattern_from_template("Error <*> occurred at <*>")
        '^Error\\s+(\\S+?)\\s+occurred\\s+at\\s+(\\S+?)$'

        >>> get_pattern_from_template("File <*> not found")
        '^File\\s+(\\S+?)\\s+not\\s+found$'

        >>> get_pattern_from_template("User <*> logged in at <*>")
        '^User\\s+(\\S+?)\\s+logged\\s+in\\s+at\\s+(\\S+?)$'
    """
    escaped = re.escape(template)
    spaced_escape = re.sub(r'\\\s+', "\\\s+", escaped)
    return "^" + spaced_escape.replace(r"<\*>", r"(\S+?)") + "$"  # Một <*> duy nhất có thể sử dụng nhiều token

def is_abstract(x, y):
    """ Xác định xem template `x` có trừu tượng hơn chuỗi `y` hay không. Hàm này kiểm tra xem chuỗi `y` có khớp với biểu thức chính quy được tạo từ template `x` hay không. Template `x` có thể chứa ký tự đại diện `<*>`, đại diện cho một hoặc nhiều ký tự không phải khoảng trắng.

    Args:
        x (str): template (template).
        y (str): Chuỗi cần kiểm tra (template hoặc message).

    Returns:
        bool: Trả về `True` nếu `x` trừu tượng hơn `y`, ngược lại trả về `False`.

    Ví dụ:
        x = "Hello <*>"
        y = "Hello world"
        is_abstract(x, y)  # Trả về True

        y = "Hi world"
        is_abstract(x, y)  # Trả về False
    """

    if y is np.nan:
        return False

    m = re.match(get_pattern_from_template(x), y)
    if m:
        return True
    else:
        return False


def common_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-otc', '--oracle_template_correction',
                        help="Set this if you want to use corrected oracle templates",
                        default=False, action='store_true')
    parser.add_argument('-full', '--full_data',
                        help="Set this if you want to test on full dataset",
                        default=False, action='store_true')
    parser.add_argument('--complex', type=int,
                        help="Set this if you want to test on complex dataset",
                        default=0)
    parser.add_argument('--frequent', type=int,
                        help="Set this if you want to test on frequent dataset",
                        default=0)
    parser.add_argument('--shot', type=int,
                        help="Set this if you want to test on complex dataset",
                        default=0)
    parser.add_argument('--example_size', type=int,
                        help="Set this if you want to test on frequent dataset",
                        default=0)    
    args = parser.parse_args()
    return args


def unique_output_dir(name):
    """ Tạo một đường dẫn thư mục đầu ra duy nhất dựa trên tên cơ sở, thời gian hiện tại và ID tiến trình.

    Args:
        name (str): Tên cơ sở của thư mục.

    Returns:
        str: Đường dẫn thư mục đầu ra duy nhất.

    Examples:
        >>> unique_output_dir("experiment")
        'experiment_result/20231005_153045_12345'
    """
    return os.path.join('{}_result'.format(name), '{}_{}'.format(datetime.now().strftime("%Y%m%d_%H%M%S"), os.getpid()))


def correct_single_template(template, user_strings=None):
    """ Áp dụng các quy tắc để xử lý và chuẩn hóa một chuỗi template. Dựa trên bài báo: "Guidelines for Assessing the Accuracy of Log Message Template Identification Techniques (2023)".
    DOI: 10.1145/3510003.3510101

    Các quy tắc được áp dụng:
        - **DS (Double Space)**: Loại bỏ khoảng trắng thừa và chuẩn hóa chuỗi thành một khoảng trắng duy nhất. `Input:  <*>` --> `Input: <*>`
        - **BL (Boolean)**: Thay thế các giá trị boolean như "true", "false" bằng ký tự đại diện `<*>`. `cancel=false` --> `cancel=<*>`
        - **US (User String)**: Thay thế các chuỗi do người dùng định nghĩa (ex, "null", "root", "admin") bằng `<*>`. `status=idle` --> `status=<*>`
        - **DG (Digit)**: Thay thế các giá trị số bằng `<*>`. `euid=0` --> `euid=<*>`
        - **PS (Path String)**: Thay thế các chuỗi dạng đường dẫn bằng `<*>`. `/lib/tmp started` --> `<*> started`
        - **WV (Word concatenated with Variable, hay Mixed Token (MT))**: Thay thế các chuỗi chứa `<*>` được nối với các ký tự khác. `python v<*>` --> `python <*>`
        - **DV (Dot-separated Variables)**: Thay thế các chuỗi dạng `<*>.<*>` bằng `<*>`. `<*>.<*> seconds` --> `<*> seconds`
        - **CV (Consecutive Variables)**: Thay thế các chuỗi `<*><*>` liên tiếp bằng `<*>`.  `value=<*><*>` --> `value=<*>`

    Args:
        template (str): Chuỗi template cần xử lý.
        user_strings (set, optional): Tập hợp các chuỗi do người dùng định nghĩa cần thay thế bằng `<*>`.

    Returns:
        str: Chuỗi template đã được xử lý và chuẩn hóa.

    Ví dụ:
        >>> template = "Received block <*> of size <*> from <*>"
        >>> correct_single_template(template)
        'Received block <*> of size <*> from <*>'

        >>> template = "/path/to/file file/path"
        >>> correct_single_template(template)
        '<*> file/path'
    
    """

    boolean = {'true', 'false'}                 # Các giá trị boolean (BL)
    default_strings = {'null', 'root', 'admin'} # Các chuỗi do người dùng định nghĩa, mặc định là 3 gtrị này (US)
    path_delimiters = {                         # Các ký tự sử dụng phân tách đường dẫn (PS)
        r'\s', r'\,', r'\!', r'\;', r'\:',
        r'\=', r'\|', r'\"', r'\'',
        r'\[', r'\]', r'\(', r'\)', r'\{', r'\}'
    }
    token_delimiters = path_delimiters.union({  # Sử dụng để phân tách các kỹ thuật rule còn lại (DG, WV, DV, CV)
        r'\.', r'\-', r'\+', r'\@', r'\#', r'\$', r'\%', r'\&',
    })

    if user_strings:
        default_strings = default_strings.union(user_strings) # Nếu có các chuỗi do người dùng định nghĩa, thêm vào danh sách mặc định

    # Áp dụng DS (Double Space)
    template = template.strip()
    template = re.sub(r'\s+', ' ', template)

    # Áp dụng PS (Path String)
    p_tokens = re.split('('+'|'.join(path_delimiters)+')', template)
    new_p_tokens = []
    for p_token in p_tokens:
        if re.match(r'^(\/[^\/]+)+$', p_token):
            p_token = '<*>'
        new_p_tokens.append(p_token)
    template = ''.join(new_p_tokens)

    # Phân tách thành các token để xủ lý các rule con lại 
    tokens = re.split('('+'|'.join(token_delimiters)+')', template)  # Chuẩn hóa token nhưng vẫn giữ lại các ký tự phân tách
    new_tokens = []
    for token in tokens:
        # Áp dụng BL (Boolean), US (User String)
        for to_replace in boolean.union(default_strings):
            if token.lower() == to_replace.lower():                 # ==> Chuyển thành chữ thường để so sánh
                token = '<*>'

        # Áp dụng DG (Digit)
        if re.match(r'^\d+$', token):
            token = '<*>'

        # Áp dụng WV (Word concatenated with Variable), hay Mixed Token (MT)
        if re.match(r'^[^\s\/]*<\*>[^\s\/]*$', token):
            if token != '<*>/<*>':  # Cần kiểm tra vì `/` không phải là ký tự phân tách
                token = '<*>'

        # Thu thập các token đã xử lý
        new_tokens.append(token)

    # Tạo ra template mới, template hoàn chỉnh với độ trừu tượng cao
    template = ''.join(new_tokens)

    # Chỉ thay thế các biến liên tiếp nếu được phân tách bằng bất kỳ phân tách nào bao gồm "." (DV)
    while True:
        prev = template
        template = re.sub(r'<\*>\.<\*>', '<*>', template)
        if prev == template:
            break

    # Chỉ thay thế các biến liên tiếp nếu không được phân tách bằng bất kỳ phân tách nào bao gồm khoảng trắng (CV)
    # NOTE: Quá trình này nên thực hiện cuối cùng để tránh thay thế các biến liên tiếp không cần thiết
    while True:
        prev = template
        template = re.sub(r'<\*><\*>', '<*>', template)
        if prev == template:
            break

    return template


def correct_templates_and_update_files(dir_path, log_file_basename, inplace=False):
    """ Cập nhật file log có cấu trúc và file template sau khi sửa các template theo luật hậu xử lý.
    - Nếu một template bị thay đổi nội dung (nhưng không thay đổi EventId), chỉ cập nhật cột `EventTemplate`.
    - Nếu nhiều template được gộp thành một (ví dụ do khái quát hóa), cập nhật cả `EventId` và `EventTemplate` trong structured log. EventId mới được chọn là EventId đầu tiên trong nhóm ban đầu để giữ đồng nhất.

    Args:
        dir_path (str): Đường dẫn tới thư mục chứa các file structured log và template.
        log_file_basename (str): Tên gốc của file log (ví dụ: `BGL_2k.log`). Hàm sẽ dùng để tìm các file liên quan như
                                 `BGL_2k.log_structured.csv`.
        inplace (bool, optional): Nếu True, các file structured log và templates sẽ được ghi đè;
                                  nếu False, sẽ tạo ra các file mới với hậu tố `_corrected.csv`.

    Returns:
        None: Hàm không trả về giá trị, 

    Examples:
        >>> File input:
            templates:
                E1, Send 123
                E2, Send 456

        >>> structured log:
                LineId, Content, EventId, EventTemplate
                1, Send 123, E1, Send 123
                2, Send 456, E2, Send 456

        Sau khi gộp và xử lý:
        >>> templates:
                E1, Send <*>
        Và:
        >>> structured log:
                1, Send 123, E1, Send <*>
                2, Send 456, E1, Send <*>
        Ví dụ:
        >>> correct_templates_and_update_files("./logs/", "BGL_2k.log", inplace=False)
    """

    # Tạo đường dẫn đến file structured log
    org_structured_log_file = os.path.join(dir_path, log_file_basename + '_structured.csv')

    # Chuyển dữ liệu từ file structured log thành DataFrame
    structured_logs_df = pd.read_csv(org_structured_log_file)

    # Trích xuất template qua biến templates_df
    # Loại bỏ các dòng trùng lặp dựa trên cột EventTemplate. Sau đó, loại bỏ các dòng có giá trị NaN trong cột EventTemplate, sử dụng dropna (điều này cần thiết vì một số công cụ như LogSig có thể tạo ra các template rỗng).
    templates_df = structured_logs_df.drop_duplicates(subset='EventTemplate').dropna(subset=['EventTemplate'])

    # Chuyển templates_df thành dictionary (key: EventId, value: EventTemplate)
    templates_dict = templates_df.set_index('EventId')['EventTemplate'].to_dict()

    # Áp dụng 8 quy tắc sửa template và gộp template
    new_templates_dict = correct_templates(templates_dict)

    # Cập nhật DataFrame structured_logs_df
    start_time = time.time()
    # Duyệt qua từng dòng trong structured_logs_df. Tìm template tương ứng với EventId trong new_templates_dict.
    for index, row in structured_logs_df.iterrows():

        is_matched = False
        for tids, template in new_templates_dict.items():
            if row['EventId'] in tids:  # Nếu tìm thấy, cập nhật EventId và EventTemplate trong DataFrame.
                structured_logs_df.at[index, 'EventId'] = tids[0]
                structured_logs_df.at[index, 'EventTemplate'] = template
                is_matched = True
                break

        # Nếu không tìm thấy, in cảnh báo với thông tin EventId và nội dung (Content) của dòng đó.
        if is_matched is False:
            print('*** WARN: No matching template; EventId:', row['EventId'], 'message:', row['Content'])

    # Cập nhật file structured log
    if inplace:  # Ghi đè file gốc
        structured_logs_df.to_csv(org_structured_log_file, index=False)
    else:  # Sử dụng tên file mới với hậu tố _corrected.csv
        structured_logs_df.to_csv(os.path.join(dir_path, log_file_basename + '_structured_corrected.csv'), index=False)

    # Cập nhật file templates
    new_templates = []
    for tids, template in new_templates_dict.items():
        new_templates.append((tids[0], template))
    new_templates = natsorted(new_templates, key=lambda x: x[0])
    new_templates_df = pd.DataFrame(new_templates, columns=['EventId', 'EventTemplate'])
    new_templates_df.to_csv(os.path.join(dir_path, log_file_basename + '_templates_corrected.csv'), index=False)

    print('Structured log and templates file update done. [Time taken: {:.3f}]'.format(time.time() - start_time))

def correct_templates(templates_dict):
    """
    Core function for template postprocessing.

    :param templates_dict: existing templates (key: EventId, value: EventTemplate)
    :return: new templates_dict (key: Tuple of EventIds, value: EventTemplate)
    """

    # templates that are affected by the post-processing
    change_count = 0
    inverse_templates_dict = {}  # key: EventTemplate, value: list of EventIds

    start_time = time.time()
    for tid, template in sorted(templates_dict.items(), key=lambda x: x[0]):  # sort to avoid non-determinism
        org_template = template
        new_template = correct_single_template(template)

        # count the number of changed templates
        if org_template != new_template:
            change_count += 1

        # update temp_templates_dict
        if new_template in inverse_templates_dict.keys():
            inverse_templates_dict[new_template].append(tid)
        else:
            inverse_templates_dict[new_template] = [tid]

    # build new_templates_dict using inverse_templates_dict
    new_templates_dict = {tuple(tids): template for template, tids in inverse_templates_dict.items()}

    end_time = time.time() - start_time
    print('\tOriginal templates:', len(templates_dict.keys()))
    print('\tTemplates after correction:', len(new_templates_dict.keys()))
    print("\tTemplates changed by correction:", change_count)
    print('Template correction done. [Time taken: {:.3f}]'.format(end_time))

    return new_templates_dict

In [ ]:
# File template_level_analysis.py trong UNLEASH

from __future__ import print_function

import os
import pandas as pd
# from unleash.evaluation.utils.common import is_abstract
from tqdm import tqdm


def evaluate_template_level(dataset, df_groundtruth, df_parsedresult, filter_templates=None):
    """
    Đánh giá mức độ chính xác của template ở mức template-level dựa trên các kết quả phân tích đã cho, bao gồm các chỉ số FTA, PTA, RTA. Cách thực hiện tương tự như tính chỉ số GA, FGA

    Args:
        dataset: Tập dữ liệu đầu vào (không được sử dụng trong hàm này).
        df_groundtruth (pd.DataFrame): DataFrame chứa các template sự kiện thực tế (groundtruth), với cột 'EventTemplate'.
        df_parsedresult (pd.DataFrame): DataFrame chứa các template sự kiện được phân tích (parsed result), với cột 'EventTemplate'.
        filter_templates (set, optional): Tập hợp các template cần lọc để đánh giá. Nếu không được cung cấp, sẽ đánh giá toàn bộ.

    Returns:
        tuple: Gồm các giá trị:
            - t1 (int): Số lượng template được nhận diện.
            - t2 (int): Số lượng template thực tế.
            - FTA (float): Giá trị F1-Score (F-Measure) của việc phân tích template.
            - PTA (float): Độ chính xác (Precision Template Accuracy).
            - RTA (float): Độ bao phủ (Recall Template Accuracy).

    Examples:
        >>> dataset = None
        >>> df_groundtruth = pd.DataFrame({'EventTemplate': ['A', 'B', 'C', None]})
        >>> df_parsedresult = pd.DataFrame({'EventTemplate': ['A', 'B', 'D', None]})
        >>> filter_templates = {'A', 'B'}
        >>> evaluate_template_level(dataset, df_groundtruth, df_parsedresult, filter_templates)
        Identify : 2, Groundtruth : 2
        PTA: 1.0000, RTA: 1.0000 FTA: 1.0000
        (2, 2, 1.0, 1.0, 1.0)
    """
    # Tương tự như quy trình tính toán chỉ số GA, nhưng có một số điểm khác nhau.
    correct_parsing_templates = 0               # Số lượng template được phân tích đúng
    if filter_templates is not None:
        filter_identify_templates = set()
    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index
    df_groundtruth = df_groundtruth.loc[null_logids]
    df_parsedresult = df_parsedresult.loc[null_logids]
    series_groundtruth = df_groundtruth['EventTemplate']
    series_parsedlog = df_parsedresult['EventTemplate']
    series_groundtruth_valuecounts = series_groundtruth.value_counts()

    df_combined = pd.concat([series_groundtruth, series_parsedlog], axis=1, keys=['groundtruth', 'parsedlog'])
    grouped_df = df_combined.groupby('parsedlog')   # Tính toán theo template của parserlog
    
    for identified_template, group in tqdm(grouped_df):
        corr_oracle_templates = set(list(group['groundtruth'])) # DS các template thực tế tương ứng với template phân tích

        if filter_templates is not None and len(corr_oracle_templates.intersection(set(filter_templates))) > 0:
            filter_identify_templates.add(identified_template)

        if corr_oracle_templates == {identified_template}:
            if (filter_templates is None) or (identified_template in filter_templates):
                correct_parsing_templates += 1

    if filter_templates is not None:
        PTA = correct_parsing_templates / len(filter_identify_templates)
        RTA = correct_parsing_templates / len(filter_templates)
    else:
        PTA = correct_parsing_templates / len(grouped_df)
        RTA = correct_parsing_templates / len(series_groundtruth_valuecounts)
    FTA = 0.0
    if PTA != 0 or RTA != 0:
        FTA = 2 * (PTA * RTA) / (PTA + RTA)
    print('PTA: {:.4f}, RTA: {:.4f} FTA: {:.4f}'.format(PTA, RTA, FTA))
    t1 = len(grouped_df) if filter_templates is None else len(filter_identify_templates)
    t2 = len(series_groundtruth_valuecounts) if filter_templates is None else len(filter_templates)
    print("Identify : {}, Groundtruth : {}".format(t1, t2))
    return t1, t2, FTA, PTA, RTA


def evaluate_template_level_lstm(dataset, df_groundtruth, df_parsedresult, filter_templates=None):
    """ Tương tự, tính toán chỉ số FTA, PTA, RTA cho các trình phân tích cú pháp dựa trên ngữ nghĩa. Quy trình hoạt động tương tự như evaluate_template_level, nhưng sử dụng một phương thức kiểm tra độ chính xác khác (correct_lstm).
    Args:
        dataset: Tập dữ liệu đầu vào (không được sử dụng trong hàm này).
        df_groundtruth (pd.DataFrame): DataFrame chứa các template sự kiện thực tế (groundtruth), với cột 'EventTemplate'.
        df_parsedresult (pd.DataFrame): DataFrame chứa các template sự kiện được phân tích (parsed result), với cột 'EventTemplate'.
        filter_templates (set, optional): Tập hợp các template cần lọc để đánh giá. Nếu không được cung cấp, sẽ đánh giá toàn bộ.

    Returns:
        tuple: Gồm các giá trị:
            - t1 (int): Số lượng template được nhận diện.
            - t2 (int): Số lượng template thực tế.
            - FTA (float): Giá trị F1-Score (F-Measure) của việc phân tích template.
            - PTA (float): Độ chính xác (Precision Template Accuracy).
            - RTA (float): Độ bao phủ (Recall Template Accuracy).
    """

    correct_parsing_templates = 0
    if filter_templates is not None:
        filter_identify_templates = set()
    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index
    df_groundtruth = df_groundtruth.loc[null_logids]
    df_parsedresult = df_parsedresult.loc[null_logids]
    series_groundtruth = df_groundtruth['EventTemplate']
    series_parsedlog = df_parsedresult['EventTemplate']
    series_groundtruth_valuecounts = series_groundtruth.value_counts()

    df_combined = pd.concat([series_groundtruth, series_parsedlog], axis=1, keys=['groundtruth', 'parsedlog'])
    grouped_df = df_combined.groupby('parsedlog')
    
    for identified_template, group in tqdm(grouped_df):
        corr_oracle_templates = set(list(group['groundtruth']))
        if filter_templates is not None and len(corr_oracle_templates.intersection(set(filter_templates))) > 0:
            filter_identify_templates.add(identified_template)
        
        if len(corr_oracle_templates) == 1 and correct_lstm(identified_template, list(corr_oracle_templates)[0]):
            if (filter_templates is None) or (list(corr_oracle_templates)[0] in filter_templates):
                correct_parsing_templates += 1

    if filter_templates is not None:
        PTA = correct_parsing_templates / len(filter_identify_templates)
        RTA = correct_parsing_templates / len(filter_templates)
    else:
        PTA = correct_parsing_templates / len(grouped_df)
        RTA = correct_parsing_templates / len(series_groundtruth_valuecounts)
    FTA = 0.0
    if PTA != 0 or RTA != 0:
        FTA = 2 * (PTA * RTA) / (PTA + RTA)
    print('PTA: {:.4f}, RTA: {:.4f} FTA: {:.4f}'.format(PTA, RTA, FTA))
    t1 = len(grouped_df) if filter_templates is None else len(filter_identify_templates)
    t2 = len(series_groundtruth_valuecounts) if filter_templates is None else len(filter_templates)
    print("Identify : {}, Groundtruth : {}".format(t1, t2))
    return t1, t2, FTA, PTA, RTA


def correct_lstm(groudtruth, parsedresult):
    """ Phương thức sử dụng để tính toán độ chính xác phân tích dành riêng cho các trình phân tích cú pháp dựa trên ngữ nghĩa. Bản chất, chỉ chỉnh sửa lại, lọc các nhiễu trong groudtruth để so sánh với parsedresult. 
    """
    tokens1 = groudtruth.split(' ')
    tokens2 = parsedresult.split(' ')
    tokens1 = ["<*>" if "<*>" in token else token for token in tokens1]
    return tokens1 == tokens2

# def compute_template_level_accuracy(num_oracle_template, comparison_results_df):
#     """
#     Tính toán độ chính xác ở cấp độ Template (template-level accuracy), dựa trên kết quả phân loại từng template thành các nhóm:

#     - **SM (Strict Match)**: Template phát hiện khớp hoàn toàn với template groundtruth.
#     - **OG (Over-Generalized)**: Template phát hiện quá khái quát, đại diện cho nhiều template groundtruth khác nhau.
#     - **UG (Under-Generalized)**: Template phát hiện quá cụ thể, chia nhỏ một template groundtruth thành nhiều cái.
#     - **MX (Mixed)**: Trường hợp template vừa bị OG vừa bị UG.

#     Các chỉ số được tính:
#     - **Precision (PTA)**: Tỷ lệ template phát hiện đúng (SM) / tổng số template được phát hiện.
#     - **Recall (RTA)**: Tỷ lệ template phát hiện đúng (SM) / tổng số template groundtruth.
#     - **F1-Score (FTA)**: Trung bình điều hòa giữa precision và recall.
#     - **OG, UG, MX**: Tỷ lệ các lỗi phát hiện template (OG, UG, MX) so với tổng số template được phát hiện.

#     Args:
#         num_oracle_template (int): Tổng số template groundtruth (được gán nhãn đúng từ ban đầu).
#         comparison_results_df (pandas.DataFrame): Bảng dữ liệu chứa các template được công cụ phát hiện. Phải có cột `type` thể hiện loại so khớp (`SM`, `OG`, `UG`, `MX`).

#     Returns:
#         tuple: Gồm các giá trị sau:
#             - f1_measure (float): F1-score của việc phát hiện template.
#             - precision (float): Precision (PTA).
#             - recall (float): Recall (RTA).
#             - over_generalized (float): Tỷ lệ OG templates.
#             - under_generalized (float): Tỷ lệ UG templates.
#             - mixed (float): Tỷ lệ MX templates.

#     Examples:
#         >>> import pandas as pd
#         >>> data = {'EventTemplate': ['T1', 'T2', 'T3', 'T4', 'T5'],
#         ...         'type': ['SM', 'OG', 'SM', 'UG', 'MX']}
#         >>> df = pd.DataFrame(data)
#         >>> compute_template_level_accuracy(5, df)
#         (0.4, 0.4, 0.4, 0.2, 0.2, 0.2)
#     """
#     count_total = float(len(comparison_results_df)) # Tổng số template được phát hiện
#     over_generalized = len(comparison_results_df[comparison_results_df.type == 'OG']) / count_total     # Tỷ lệ OG templates
#     under_generalized = len(comparison_results_df[comparison_results_df.type == 'UG']) / count_total     # Tỷ lệ UG templates
#     mixed = len(comparison_results_df[comparison_results_df.type == 'MX']) / count_total
#     return over_generalized, under_generalized, mixed

In [1]:
# File evaluator_main.py trong UNLEASH

import json
import os
import re
import time
import csv
from multiprocessing import Process

from tqdm import tqdm
import pandas as pd

# from unleash.evaluation.utils.common import correct_templates_and_update_files
# from unleash.evaluation.utils.GA_calculator import evaluate
# from unleash.evaluation.utils.template_level_analysis import evaluate_template_level, evaluate_template_level_lstm
# from unleash.evaluation.utils.PA_calculator import calculate_parsing_accuracy, calculate_parsing_accuracy_lstm
# from .post_process import correct_single_template

# TIMEOUT = 3600 * 12  # log template identification timeout (sec)
TIMEOUT = 3600 * 48  # log template identification timeout (sec)


def prepare_results(output_dir):
    """ Phương thức chuẩn bị file kết quả trong thư mục đầu ra. Hàm này thực hiện các bước sau:
    1. Kiểm tra xem thư mục đầu ra (`output_dir`) có tồn tại hay không. Nếu không, tạo thư mục mới.
    2. Kiểm tra xem tệp kết quả `'parsing_accuracy.csv'` đã tồn tại trong thư mục hay chưa. Nếu chưa, tạo tệp mới và ghi dòng tiêu đề vào tệp CSV. Trả về tên tệp kết quả.

    Args:
        - output_dir (str): Đường dẫn đến thư mục đầu ra.

    Returns:
        - str: Tên tệp kết quả (`'parsing_accuracy.csv'`).
    """
    if not os.path.exists(output_dir):
        # Tạo mới nếu không tồn tại
        os.makedirs(output_dir)

    # Tạo tệp kết quả nếu chưa tồn tại
    result_file = 'parsing_accuracy.csv'
    if not os.path.exists(os.path.join(output_dir, result_file)):
        with open(os.path.join(output_dir, result_file), 'w') as csv_file:
            fw = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            fw.writerow(['Dataset', 'traning_time', 'parsing_time', 'identified_templates',
                        'ground_templates', 'GA', 'PA', 'FGA', 'PTA', 'RTA', 'FTA']) # Tiêu đề cột trong tệp CSV

    return result_file


def correct_template_general(template):
    """ Phương thức thực hiện chỉnh sửa lỗi template chuẩn theo 2 quy tắc chính (DV, CV).
    Args:
        template (str): Chuỗi template cần xử lý.
    Returns:
        str: Chuỗi template đã được xử lý và chuẩn hóa.
    """
    # Chỉ thay thế các biến liên tiếp nếu được phân tách bằng bất kỳ phân tách nào bao gồm "." (DV)
    while True:
        prev = template
        template = re.sub(r'<\*>\.<\*>', '<*>', template)
        if prev == template:
            break

    # Thay thế các biến liên tiếp (CV)
    # NOTE: Cần phải thực hiện cuối cùng
    while True:
        prev = template
        template = re.sub(r'<\*><\*>', '<*>', template)
        template = re.sub(r'<\*>\:<\*>', '<*>', template)
        template = re.sub(r'<\*> <\*>', '<*>', template)
        if prev == template:
            break
        
    return template

def align_with_null_values(groudtruth_row):
    """ Căn chỉnh các giá trị null trong template sự kiện với nội dung thực tế. Phương thức này giúp đảm bảo rằng các template được căn chỉnh chính xác với nội dung, đặc biệt trong các trường hợp có placeholder (<*>) hoặc các giá trị null.
    Args:
        groudtruth_row (dict): Một dictionary chứa:
            - 'Content' (str): Chuỗi nội dung thực tế.
            - 'EventTemplate' (str): Chuỗi template sự kiện.

    Returns:
        str: Chuỗi template sự kiện đã được căn chỉnh với nội dung thực tế.

    Examples:
        >>> groudtruth_row = {
        ...     'Content': 'User logged in from 192.168.1.1',
        ...     'EventTemplate': 'User <*> in from <*>'
        ... }
        >>> align_with_null_values(groudtruth_row)
        'User <*> in from <*>'

        >>> groudtruth_row = {
        ...     'Content': 'File uploaded successfully',
        ...     'EventTemplate': 'File <*> successfully'
        ... }
        >>> align_with_null_values(groudtruth_row)
        'File <*> successfully'
    """

    log = groudtruth_row['Content']
    template = groudtruth_row['EventTemplate']

    # Tạo biểu thức chính quy theo template để so khớp với log
    pattern_parts = template.split("<*>")
    pattern_parts_escaped = [re.escape(part) for part in pattern_parts]
    regex_pattern = "(.*?)".join(pattern_parts_escaped)
    regex = "^" + regex_pattern + "$"  
    matches = re.search(regex, log)

    if matches == None:     # Nếu không khớp với chuỗi ban đầu, trả về template gốc
        return template

    parts = []
    for index, part in enumerate(template.split("<*>")):
        parts.append(part)
        if index < len(matches.groups()):
            if matches.groups()[index] == '':
                parts.append('')
            else:
                parts.append('<*>')
    return ''.join(parts)


def is_file_empty(file_path):
    """ Phương thức kiểm tra xem tệp có rỗng hay không.

    Args:
        file_path (str): Đường dẫn đến tệp cần kiểm tra.

    Returns:
        bool: Trả về True nếu tệp rỗng, ngược lại trả về False.
    """
    with open(file_path, 'r') as file:
        content = file.read()
        return len(content) == 0


def evaluator(
        dataset,
        input_dir,
        output_dir,
        log_file,
        result_file,
        lstm=False
):
    """ Phương thức đánh giá hiệu suất phân tích log dựa trên file đầu ra đã phân tích và dữ liệu ground truth. Tuy nhiên, đây chỉ sử dụng cho UNLEASH, chưa tổng quát như Loghub 2.0. File này chưa có tùy chọn chỉnh sửa lại các lỗi trong template, tùy chọn đánh giá trên các tập template cụ thể, ...

    Args:
        dataset (str): Tên của tập dữ liệu log.
        input_dir (str): Thư mục chứa log gốc và ground truth.
        output_dir (str): Thư mục chứa kết quả phân tích (parsed log).
        log_file (str): Tên file log cần đánh giá (không bao gồm phần structured).
        result_file (str): Tên file kết quả tổng hợp sẽ được ghi.
        lstm (bool, optional): Nếu True, sử dụng phương pháp đánh giá có liên quan đến LSTM.

    Returns:
        None. Kết quả được ghi trực tiếp vào `result_file`.

    Examples:
        >>> evaluator(
                dataset="HDFS",
                input_dir="logs/raw",
                output_dir="logs/parsed",
                log_file="HDFS.log",
                result_file="summary.csv",
                lstm=False
            )
    """

    print('\n=== Evaluation on %s ===' % dataset)
    # Xác định đường dẫn đến file log gốc và file ground truth
    indir = os.path.join(input_dir, os.path.dirname(log_file))
    log_file_basename = os.path.basename(log_file)
    
    groundtruth = os.path.join(indir, log_file_basename + '_structured.csv')
    parsedresult = os.path.join(output_dir, log_file_basename + '_structured.csv')

    # print(parsedresult)
    if not os.path.exists(parsedresult) or is_file_empty(parsedresult): # Xử lý nếu không tồn tại
        print("No output file generated.")
        result = dataset + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + '\n'
                # "{:.1f}".format(GA_end_time) + ',' + \
                # "{:.1f}".format(PA_end_time) + ',' + \
                # "{:.1f}".format(TA_end_time) + ',' + \

        with open(os.path.join(output_dir, result_file), 'a') as summary_file:
            summary_file.write(result)
        return   

    parsedresult = pd.read_csv(parsedresult, dtype=str)
    parsedresult.fillna("", inplace=True)               # Thay thế giá trị NaN bằng chuỗi rỗng
    groundtruth = pd.read_csv(groundtruth, dtype=str)

    # Chuẩn hóa các template bằng phương thức `align_with_null_values` và `correct_template_general`
    tqdm.pandas()
    print("Start to align with null values")
    groundtruth['EventTemplate'] = groundtruth.progress_apply(align_with_null_values, axis=1)
    groundtruth['EventTemplate'] = groundtruth['EventTemplate'].map(correct_template_general)
    parsedresult['EventTemplate'] = parsedresult.progress_apply(align_with_null_values, axis=1)

    filter_templates = None
    print("Start compute grouping accuracy")
    # Tính toán chỉ số GA bằng phương thức `evaluate` trong GA_calculator.py
    start_time = time.time()
    GA, FGA = evaluate(groundtruth, parsedresult, filter_templates)

    GA_end_time = time.time() - start_time
    print('Grouping Accuracy calculation done. [Time taken: {:.3f}]'.format(GA_end_time))

    # Tính toán chỉ số PA bằng phương thức `calculate_parsing_accuracy` trong PA_calculator.py
    start_time = time.time()
    if lstm == True:
        PA = calculate_parsing_accuracy_lstm(groundtruth, parsedresult, filter_templates)
        print("Finish calculate_parsing_accuracy_lstm")
    else:
        PA = calculate_parsing_accuracy(groundtruth, parsedresult, filter_templates)
    PA_end_time = time.time() - start_time
    print('Parsing Accuracy calculation done. [Time taken: {:.3f}]'.format(PA_end_time))

    # Tính toán chỉ số FTA, PTA, RTA bằng phương thức `evaluate_template_level` trong template_level_analysis.py
    start_time = time.time()
    if lstm == True:
        tool_templates, ground_templates, FTA, PTA, RTA = evaluate_template_level_lstm(dataset, groundtruth, parsedresult, filter_templates)
    else:
        tool_templates, ground_templates, FTA, PTA, RTA = evaluate_template_level(dataset, groundtruth, parsedresult, filter_templates)
    TA_end_time = time.time() - start_time
    print('Template-level accuracy calculation done. [Time taken: {:.3f}]'.format(TA_end_time))

    # Đọc thời gian phân tích và thời gian huấn luyện 
    time_cost_file = os.path.join(output_dir, 'time_cost.json')
    parsing_time, training_time = 0, 0
    if os.path.exists(time_cost_file):
        with open(time_cost_file, 'r') as file:
            time_table = json.load(file)
            training_time = time_table[dataset]['TrainingTime']
            parsing_time = time_table[dataset]['ParsingTime']

    # Ghi kết quả vào result_file
    result = dataset + ',' + \
             "{:.3f}".format(training_time) + ',' + \
             "{:.3f}".format(parsing_time) + ',' + \
             str(tool_templates) + ',' + \
             str(ground_templates) + ',' + \
             "{:.3f}".format(GA) + ',' + \
             "{:.3f}".format(PA) + ',' + \
             "{:.3f}".format(FGA) + ',' + \
             "{:.3f}".format(PTA) + ',' + \
             "{:.3f}".format(RTA) + ',' + \
             "{:.3f}".format(FTA) + '\n'
             # "{:.1f}".format(GA_end_time) + ',' + \
             # "{:.1f}".format(PA_end_time) + ',' + \
             # "{:.1f}".format(TA_end_time) + ',' + \

    with open(os.path.join(output_dir, result_file), 'a') as summary_file:
        summary_file.write(result)

In [ ]:
# File oracle_template_correction.py trong UNLEASH

from __future__ import print_function
import os
# from evaluation.utils.common import correct_templates_and_update_files, datasets

def main():
    # Thực hiện quy trình xử lý và cập nhật các file log có cấu trúc và file template cho từng loại dataset.
    for system in datasets:
        print('-' * 70)
        print(system)
        dir_path = os.path.join('..', 'logs', system)
        log_file_basename = system + '_2k.log'
        correct_templates_and_update_files(dir_path, log_file_basename, inplace=False) # Cập nhật lại và ghi ra file mới

if __name__ == '__main__':
    main()

In [ ]:
# File post_process.py trong UNLEASH
import regex as re

param_regex = [
    r'{([ :_#.\-\w\d]+)}', # Tìm các chuỗi nằm trong cặp dấu ngoặc nhọn {}. Ex: text = "This is a {template_1} and another {example:template#2}." --> re.findall(pattern, text) = ['template_1', 'example:template#2']
    r'{}'                  # Tìm các cặp dấu ngoặc nhọn {} rỗng, không chứa bất kỳ nội dung nào bên trong. Ex: text = "This is a {} and another {}." --> re.findall(pattern, text) = ['{}', '{}']
] # Biến này không được sử dụng

def correct_single_template(template, user_strings=None):
    """ Áp dụng các quy tắc để xử lý và chuẩn hóa một chuỗi template. Phương thức này sử dụng thêm các trường hợp liên tiếp khác ngoài CV, DV để chỉnh sửa lỗi template một cách toàn diện hơn

    Args:
        template (str): Chuỗi template cần xử lý.
        user_strings (set, optional): Tập hợp các chuỗi do người dùng định nghĩa cần thay thế bằng `<*>`.

    Returns:
        str: Chuỗi template đã được xử lý và chuẩn hóa.
    """

    boolean = {}
    default_strings = {}
    path_delimiters = {                         # Dấu phân tách sử dụng cho đường dẫn (PS)
        r'\s', r'\,', r'\!', r'\;', r'\:',
        r'\=', r'\|', r'\"', r'\'',
        r'\[', r'\]', r'\(', r'\)', r'\{', r'\}'
    }
    token_delimiters = path_delimiters.union({  # Các dấu phân tách sử dụng cho các trường hợp còn lại
        r'\.', r'\-', r'\+', r'\@', r'\#', r'\$', r'\%', r'\&',
    })

    if user_strings:
        default_strings = default_strings.union(user_strings)

    # Áp dụng DS
    template = template.strip()
    template = re.sub(r'\s+', ' ', template)

    # Áp dụng PS, tuy nhiên trong phuơng thức này không sử dụng đến PS
    # p_tokens = re.split('(' + '|'.join(path_delimiters) + ')', template)
    # new_p_tokens = []
    # for p_token in p_tokens:
        # if re.match(r'^(\/[^\/]+)+$', p_token):
            # p_token = '<*>'
        # new_p_tokens.append(p_token)
    # template = ''.join(new_p_tokens)

    # Áp dụng các quy tắc còn lại
    tokens = re.split('(' + '|'.join(token_delimiters) + ')', template)  # Chuẩn hóa token nhưng vẫn giữ lại dấu phân tách
    new_tokens = []
    for token in tokens:
        # Áp dụng BL, US, trong phương thức này không áp dụng
        # for to_replace in boolean.union(default_strings):
            # if token.lower() == to_replace.lower():
                # token = '<*>'

        # Áp dụng DG
        if re.match(r'^\d+$', token):
            token = '<*>'

        # Áp dụng WV
        if re.match(r'^[^\s\/]*<\*>[^\s\/]*$', token):
            if token != '<*>/<*>':  # need to check this because `/` is not a deliminator
                token = '<*>'

        # Thu thập kết quả
        new_tokens.append(token)

    # Tạo template mới từ các token đã xử lý
    template = ''.join(new_tokens)

    # Áp dụng DV
    while True:
        prev = template
        template = re.sub(r'<\*>\.<\*>', '<*>', template)
        if prev == template:
            break

    # Áp dụng CV
    # NOTE: Thực hiện cuối cùng
    while True:
        prev = template
        template = re.sub(r'<\*><\*>', '<*>', template)
        if prev == template:
            break
    
    # Đây là các trường hợp khác ngoài CV, DV, biến thể của CV:
    while " #<*># " in template:
        template = template.replace(" #<*># ", " <*> ")

    while " #<*> " in template:
        template = template.replace(" #<*> ", " <*> ")

    while "<*>:<*>" in template:
        template = template.replace("<*>:<*>", "<*>")

    while "<*>#<*>" in template:
        template = template.replace("<*>#<*>", "<*>")

    while "<*>/<*>" in template:
        template = template.replace("<*>/<*>", "<*>")

    while "<*>@<*>" in template:
        template = template.replace("<*>@<*>", "<*>")

    while "<*>.<*>" in template:
        template = template.replace("<*>.<*>", "<*>")

    # while "<*>,<*>" in template:
    #     template = template.replace("<*>,<*>", "<*>")

    while ' "<*>" ' in template:
        template = template.replace(' "<*>" ', ' <*> ')

    while " '<*>' " in template:
        template = template.replace(" '<*>' ", " <*> ")

    while "<*><*>" in template:
        template = template.replace("<*><*>", "<*>")

    # while "[<*>]" in template:
    #     template = template.replace("[<*>]", "<*>")

    # while "(<*>)" in template:
    #     template = template.replace("(<*>)", "<*>")

    # while "{<*>}" in template:
    #     template = template.replace("{<*>}", "<*>")    

    # while "<*> <*>" in template:
    #     template = template.replace("<*> <*>", "<*>")
    return template

In [ ]:
# File postprocess.py trong UNLEASH
import pandas as pd
import numpy as np

def post_average(metric_file):
    """ Thêm dòng trung bình vào file chỉ số đánh giá, sau đó chuyển vị bảng.

    Args:
        metric_file (str): Đường dẫn đến file CSV chứa các chỉ số đánh giá (metrics).
                           File cần có cột 'Dataset' và các cột số để tính trung bình.

    Returns:
        None. Hàm này sẽ ghi đè file gốc với bảng đã được cập nhật.
    Examples:
        Với file `metrics.csv` như sau:

        >>> Dataset,Accuracy,Precision,Recall
        ... HDFS,0.85,0.87,0.83
        ... BGL,0.88,0.89,0.86
        ... HDFS,0.85,0.87,0.83  # dòng trùng sẽ bị loại bỏ

        Sau khi chạy `post_average("metrics.csv")`, file sẽ thành:
        >>> Dataset,HDFS,BGL,Average
        ... Accuracy,0.85,0.88,0.865
        ... Precision,0.87,0.89,0.88
        ... Recall,0.83,0.86,0.845
    """
    df = pd.read_csv(metric_file, index_col=False)
    df = df.drop_duplicates(['Dataset'])                            # Xóa các dòng trùng lặp dựa trên cột 'Dataset'
    mean_row = df.select_dtypes(include=[np.number]).mean().round(3)# Tính toán giá trị trung bình, làm tròn 3 chữ số
    new_row = pd.DataFrame([['Average']], columns=['Dataset']).join(pd.DataFrame([mean_row.values], columns=mean_row.index))                # Tạo một DataFrame mới với giá trị trung bình
    df = pd.concat([df, new_row], ignore_index=True)                # Thêm dòng "Average" vào cuối bảng.
    df.to_csv(metric_file, index=False)                             # Ghi đè bảng đã cập nhật vào file gốc.
    
    # Mở lại file và chuyển vị (transpose) bảng (dòng ↔ cột), sau đó lưu lại.
    df = pd.read_csv(metric_file)
    transposed_df = df.transpose()
    transposed_df.to_csv(metric_file)                               

#### **Đến các file trong Loghub 2.0**

In [ ]:
# File postprocess.py trong Loghub 2.0
import pandas as pd
import numpy as np

def post_average(metric_file, tech, complex, frequent):
    """ Tính trung bình các chỉ số đánh giá từ file CSV, thêm dòng 'Average', lưu vào file kết quả theo đường dẫn được xác định dựa vào tham số, và chuyển vị bảng kết quả. Phương thức này mở rộng hơn và toàn diện hơn.

    Args:
        metric_file (str): Đường dẫn file CSV chứa metric (có cột 'Dataset').
        tech (str): Tên kỹ thuật/thuật toán (vd: 'Drain', 'IPLoM',...).
        complex (int): Cờ cho biết có đang xử lý dữ liệu log phức tạp không.
                       (≠ 0 → ghi vào thư mục 'complex/')
        frequent (int): Cờ cho biết có đang xử lý lọc theo tần suất không.
                        (≠ 0 → ghi vào thư mục 'frequent/')

    Returns:
        - File kết quả CSV lưu tại:
            '../../result/{tech}.csv' (mặc định)
            '../../result/complex/{tech}.csv' nếu `complex` ≠ 0
            '../../result/frequent/{tech}.csv' nếu `frequent` ≠ 0
        - Bảng kết quả được chuyển vị để dễ quan sát (dataset → cột).

    Examples:
        Giả sử file `Drain.csv` trong `metrics/` có nội dung:
        >>> Dataset,Accuracy,Precision
        ... HDFS,0.85,0.87
        ... BGL,0.88,0.89
        
        Sau khi chạy:
        >>> post_average("metrics/Drain.csv", "Drain", 0, 0)

        → File `../../result/Drain.csv` sẽ thành:
        >>> ,0,1,2
        ... Dataset,HDFS,BGL,Average
        ... Accuracy,0.85,0.88,0.865
        ... Precision,0.87,0.89,0.88
    """
    df = pd.read_csv(metric_file, index_col=False)
    df = df.drop_duplicates(['Dataset'])                # Loại bỏ các dòng trùng nhau theo tên Dataset.
    
    # Tính trung bình các cột kiểu số (float/int), làm tròn 3 chữ số, để chính xác hơn
    mean_row = df.select_dtypes(include=[np.number]).mean().round(3)
    new_row = pd.DataFrame([['Average']], columns=['Dataset']).join(pd.DataFrame([mean_row.values], columns=mean_row.index))                                            # Tạo một DataFrame mới chứa giá trị trung bình.
    df = pd.concat([df, new_row], ignore_index=True)   # Thêm dòng 'Average' vào cuối bảng.
    
    # Gán đường dẫn lưu kết quả:
    output_path = f"../../result/{tech}.csv"
    if complex != 0:
        output_path = f"../../result/complex/{tech}.csv"
    if frequent != 0:
        output_path = f"../../result/frequent/{tech}.csv"
    df.to_csv(output_path, index=False)                # Ghi bảng kết quả vào file CSV mới.
    
    # Mở lại file, chuyển vị (transpose), rồi lưu lại lần nữa: Dòng trở thành cột (và ngược lại).
    df = pd.read_csv(output_path)
    transposed_df = df.transpose()
    transposed_df.to_csv(output_path)

In [ ]:
# File evaluator_main.py trong Loghub 2.0
import os
import time
import csv
import chardet
from multiprocessing import Process
# from logparser.utils.evaluator import evaluate
# from evaluation.utils.template_level_analysis import evaluate_template_level, evaluate_template_level_lstm
# from evaluation.utils.PA_calculator import calculate_parsing_accuracy, calculate_parsing_accuracy_lstm
import pandas as pd

# TIMEOUT = 3600 * 12  # log template identification timeout (sec)
TIMEOUT = 3600 * 12  # log template identification timeout (sec)


def prepare_results(output_dir, otc, complex, frequent):
    """ Chuẩn bị tệp CSV để lưu trữ kết quả phân tích. Phương thức này kiểm tra và tạo thư mục đầu ra nếu chưa tồn tại, sau đó tạo một tệp CSV với tiêu đề (header) được xác định trước.

    Args:
        output_dir (str): Đường dẫn đến thư mục đầu ra nơi tệp CSV sẽ được lưu.
        otc (bool): Giá trị boolean để chỉ định một thuộc tính cụ thể (ví dụ: có bật OTC hay không).
        complex (bool): Giá trị boolean để chỉ định tính phức tạp.
        frequent (bool): Giá trị boolean để chỉ định tần suất.

    Returns:
        str: Tên tệp CSV đã được tạo.

    Examples:
        >>> prepare_results("output", True, False, True)
        'summary_[otc=True,complex=0,frequent=1].csv'

        >>> prepare_results("results", False, True, False)
        'summary_[otc=False,complex=1,frequent=0].csv'
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    result_file = 'summary_[otc={},complex={},frequent={}].csv'.format(str(otc), str(int(complex)), str(int(frequent)))
    with open(os.path.join(output_dir, result_file), 'w') as csv_file:
        fw = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        # fw.writerow(['Dataset', 'GA_time', 'PA_time', 'TA_time', 'parse_time', 'identified_templates',
        #              'ground_templates', 'GA', 'PA', 'FTA', 'PTA', 'RTA', 'OG', 'UG', 'MX'])
        fw.writerow(['Dataset', 'parse_time', 'identified_templates',
                     'ground_templates', 'GA', 'PA', 'FGA', 'PTA', 'RTA', 'FTA'])

    return result_file


def is_file_empty(file_path):
    """ Phương thức kiểm tra xem tệp có rỗng hay không.

    Args:
        file_path (str): Đường dẫn đến tệp cần kiểm tra.

    Returns:
        bool: Trả về True nếu tệp rỗng, ngược lại trả về False.
    """
    with open(file_path, 'r') as file:
        content = file.read()
        return len(content) == 0


def evaluator(
        dataset,
        input_dir,
        output_dir,
        log_file,
        LogParser,
        param_dict,
        otc,
        complex,
        frequent,
        result_file,
        lstm=False
):
    """
    Hàm đánh giá hiệu suất phân tích log dựa trên các chỉ số: GA, PA, FTA, RTA, PTA. Đây là hàm đánh giá chung nhất, tổng quát nhất, có thể sử dụng để đánh giá cho các trình phân tích log khác nhau.

    Args:
        dataset (str): Tên tập dữ liệu log (ví dụ: HDFS, BGL).
        input_dir (str): Thư mục chứa log và ground truth.
        output_dir (str): Thư mục chứa kết quả phân tích.
        log_file (str): Tên file log gốc.
        LogParser (class): Class parser (ví dụ Drain) sẽ được khởi tạo để thực thi phân tích log.
        param_dict (dict): Từ điển tham số khởi tạo cho parser.
        otc (bool): Nếu True, sử dụng ground truth có sửa template theo oracle.
        complex (int): Chế độ lọc mẫu phức tạp (0: không lọc, 1: đơn giản, 2: vừa, 3: phức tạp).
        frequent (int): Chế độ lọc mẫu theo tần suất (0: không lọc, >0: ít phổ biến, <0: phổ biến).
        result_file (str): Tên file CSV chứa kết quả đánh giá.
        lstm (bool, optional): Nếu True, dùng phương pháp đánh giá template có liên quan đến LSTM.

    Returns:
        None. Kết quả đánh giá được ghi vào file `result_file` trong thư mục output.

    Examples:
        >>> evaluator(
                dataset='HDFS',
                input_dir='logs/raw',
                output_dir='logs/parsed',
                log_file='HDFS.log',
                LogParser=Drain,
                param_dict={"log_format": "<Date> <Time> <Pid> <Level> <Component>: <Content>"},
                otc=True,
                complex=2,
                frequent=10,
                result_file='summary.csv',
                lstm=False
            )
    """
    print('\n=== Evaluation on %s ===' % dataset)
    indir = os.path.join(input_dir, os.path.dirname(log_file))
    log_file_basename = os.path.basename(log_file)
    
    # Nếu có otc, sử dụng tệp log đã được chỉnh sửa
    if otc:
        # Sử dụng tệp log đã được chỉnh sửa (corrected oracle templates)
        groundtruth = os.path.join(indir, log_file_basename + '_structured_corrected.csv')
    else:
        groundtruth = os.path.join(indir, log_file_basename + '_structured.csv')

    parsedresult = os.path.join(output_dir, log_file_basename + '_structured.csv')
    
    # Xác định template bằng cách sử dụng các trình phân tích (ex, Drain)
    start_time = time.time()
    if LogParser != None:
        print("start parsing.")
        parser = LogParser(**param_dict)
        p = Process(target=parser.parse, args=(log_file_basename,)) # Tạo tiến trình riêng biệt để thực hiện phân tích
        p.start()
        p.join(timeout=TIMEOUT)     # Nếu tiến trình hoàn thành trong thời gian cho phép, tiếp tục. Nếu không, tiếp tục dòng sau kiểm tra p.is_alive()
        if p.is_alive():            # Nếu parser chạy quá lâu thì huỷ
            print('*** TIMEOUT for Template Identification')
            p.terminate()
            with open(parsedresult, 'w') as fw:
                pass                # Không ghi kết quả
            return
        print("end parsing.")
        parse_time = time.time() - start_time  # end_time là thời gian trên đồng hồ tính bằng giây
    else:
        parse_time = -1
    print("parsing time: ", parse_time)

    # Nếu không có tệp kết quả phân tích hoặc tệp rỗng, ghi lại kết quả vào tệp CSV và thoát
    if not os.path.exists(parsedresult) or is_file_empty(parsedresult):
        print("No output file generated.")
        result = dataset + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + ',' + \
                "None" + '\n'
                # "{:.1f}".format(GA_end_time) + ',' + \
                # "{:.1f}".format(PA_end_time) + ',' + \
                # "{:.1f}".format(TA_end_time) + ',' + \

        with open(os.path.join(output_dir, result_file), 'a') as summary_file:
            summary_file.write(result)
        return   


    filter_templates = None
    # Lọc theo độ phức tạp (complexity) của template theo bài báo "Loghub 2.0"
    if complex != 0:
        print("Evaluate on complex mode: ", complex)
        template_file = os.path.join(indir, log_file_basename + '_templates.csv')
        df = pd.read_csv(template_file)
        if complex == 1:
            df = df[df['EventTemplate'].str.count('<*>') == 0]
        if complex == 2:
            df = df[(df['EventTemplate'].str.count('<*>') >= 1) & (df['EventTemplate'].str.count('<*>') <= 4)]
        if complex == 3:
            df = df[df['EventTemplate'].str.count('<*>') >= 5]
        filter_templates = df['EventTemplate'].tolist()
    
    # Lọc theo tần suất (frequency) của template theo bài báo "Loghub 2.0"
    if frequent != 0:
        print("Evaluate on frequent mode: ", frequent)
        template_file = os.path.join(indir, log_file_basename + '_templates.csv')
        df = pd.read_csv(template_file)
        df_sorted = df.sort_values('Occurrences')                  # Các template ít xuất hiện → nằm ở đầu danh sách.
        if frequent > 0:
            n = int(len(df_sorted) / 100.0 * frequent)
            filter_templates = df_sorted['EventTemplate'].tolist()[:n] # lấy n template đầu tiên (ít xuất hiện nhất).
        else:
            n = len(df_sorted) - int(len(df_sorted) / 100.0 * -frequent)
            filter_templates = df_sorted['EventTemplate'].tolist()[n:] # lấy n template cuối cùng (xuất hiện nhiều nhất).

    if filter_templates != None:
        print("length of filter templates: ", len(filter_templates))

    # Dừng nếu không có mẫu nào sau khi lọc
    if filter_templates != None and len(filter_templates) == 0: 
        return   

    parsedresult = pd.read_csv(parsedresult, dtype=str)
    parsedresult.fillna("", inplace=True)
    groundtruth = pd.read_csv(groundtruth, dtype=str)


    print("Start compute grouping accuracy")
    # Tính toán chỉ số GA bằng phương thức `evaluate` trong GA_calculator.py
    start_time = time.time()
    GA, FGA = evaluate(groundtruth, parsedresult, filter_templates)

    GA_end_time = time.time() - start_time
    print('Grouping Accuracy calculation done. [Time taken: {:.3f}]'.format(GA_end_time))

    # Tính toán chỉ số PA bằng phương thức `calculate_parsing_accuracy` trong PA_calculator.py
    start_time = time.time()
    if lstm == True:
        PA = calculate_parsing_accuracy_lstm(groundtruth, parsedresult, filter_templates)
        print("Finish calculate_parsing_accuracy_lstm")
    else:
        PA = calculate_parsing_accuracy(groundtruth, parsedresult, filter_templates)
    PA_end_time = time.time() - start_time
    print('Parsing Accuracy calculation done. [Time taken: {:.3f}]'.format(PA_end_time))

    # Tính toán chỉ số FTA, PTA, RTA bằng phương thức `evaluate_template_level` trong template_level_analysis.py
    start_time = time.time()
    if lstm == True:
        tool_templates, ground_templates, FTA, PTA, RTA = evaluate_template_level_lstm(dataset, groundtruth, parsedresult, filter_templates)
    else:
        tool_templates, ground_templates, FTA, PTA, RTA = evaluate_template_level(dataset, groundtruth, parsedresult, filter_templates)
    TA_end_time = time.time() - start_time
    print('Template-level accuracy calculation done. [Time taken: {:.3f}]'.format(TA_end_time))

    result = dataset + ',' + \
             "{:.2f}".format(parse_time) + ',' + \
             str(tool_templates) + ',' + \
             str(ground_templates) + ',' + \
             "{:.3f}".format(GA) + ',' + \
             "{:.3f}".format(PA) + ',' + \
             "{:.3f}".format(FGA) + ',' + \
             "{:.3f}".format(PTA) + ',' + \
             "{:.3f}".format(RTA) + ',' + \
             "{:.3f}".format(FTA) + '\n'

    with open(os.path.join(output_dir, result_file), 'a') as summary_file:
        summary_file.write(result)

In [ ]:
# File overall_evaluate.py trong Loghub 2.0
import pandas as pd
import scipy.special
from nltk.metrics.distance import edit_distance
from sklearn.metrics import accuracy_score
import numpy as np
import os
from IPython import embed

def evaluate(dataset, output_path, groundtruth, parsedresult, result_path):
    """ Hàm đánh giá độ chính xác của kết quả phân tích log bằng cách so sánh với dữ liệu ground truth.
    
    Ghi chú:
        - Hàm tính toán độ chính xác nhóm (GA), độ chính xác phân tích (PA), độ chính xác template (FTA, PTA, RTA).
        - Lọc bỏ các dòng log không có `EventTemplate` trong groundtruth.
    
    Args:
        dataset (str): Tên tập dữ liệu đang được đánh giá.
        output_path (str): Thư mục chứa kết quả đầu ra.
        groundtruth (str): Đường dẫn đến file log đã được cấu trúc hóa chuẩn (ground truth).
        parsedresult (str): Đường dẫn đến file log đã được phân tích từ công cụ log parser.
        result_path (str): Đường dẫn file kết quả nơi ghi lại các chỉ số đánh giá.

    Returns:
        None: Kết quả sẽ được in ra màn hình và ghi vào file `result_path`.

    Examples:
        >>> evaluate(
            dataset='HDFS',
            output_path='./output',
            groundtruth='./data/HDFS/HDFS_2k_structured.csv',
            parsedresult='./output/HDFS_2k_structured.csv',
            result_path='./output/result.csv'
        )
    """
    
    df_groundtruth = pd.read_csv(groundtruth)
    df_parsedlog = pd.read_csv(parsedresult, index_col=False)

    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index
    
    df_groundtruth = df_groundtruth.loc[null_logids]
    df_parsedlog = df_parsedlog.loc[null_logids]
    
    
    GA, FGA = get_group_accuracy(df_groundtruth['EventTemplate'], df_parsedlog['EventTemplate'])

    correctly_parsed_messages = df_parsedlog[['EventTemplate']].eq(df_groundtruth[['EventTemplate']]).values.sum()
    total_messages = len(df_parsedlog[['Content']])

    PA = float(correctly_parsed_messages) / total_messages

    tool_templates, ground_templates, FTA, PTA, RTA = evaluate_template_level(
        dataset=dataset,
        groundtruth=groundtruth,
        parsedresult=parsedresult,
        output_dir=output_path
    )

    result = dataset + ',' + \
             str(tool_templates) + ',' + \
             str(ground_templates) + ',' + \
             "{:.3f}".format(GA) + ',' + \
             "{:.3f}".format(FGA) + ',' + \
             "{:.3f}".format(PA) + ',' + \
             "{:.3f}".format(FTA) + ',' + \
             "{:.3f}".format(PTA) + ',' + \
             "{:.3f}".format(RTA) + '\n'
                 
    print(result)

    with open(result_path, 'a') as summary_file:
        summary_file.write(result)


def get_group_accuracy(series_groundtruth, series_parsedlog, debug=False):
    """ Đánh giá mức độ chính xác của template ở mức template-level dựa trên các kết quả phân tích đã cho, bao gồm các chỉ số FTA, PTA, RTA. Cách thực hiện tương tự như tính chỉ số GA, FGA. 
    """
    series_parsedlog_valuecounts = series_parsedlog.value_counts()
    accurate_events = 0  
    accurate_templates = 0

    for parsed_eventId in series_parsedlog_valuecounts.index:
        logIds = series_parsedlog[series_parsedlog == parsed_eventId].index
        series_groundtruth_logId_valuecounts = series_groundtruth[logIds].value_counts()
        error_eventIds = (parsed_eventId, series_groundtruth_logId_valuecounts.index.tolist())
        error = True
        if series_groundtruth_logId_valuecounts.size == 1:
            groundtruth_eventId = series_groundtruth_logId_valuecounts.index[0]
            if logIds.size == series_groundtruth[series_groundtruth == groundtruth_eventId].size:
                accurate_events += logIds.size
                accurate_templates += 1
                error = False
        if error and debug:
            print('(parsed_eventId, groundtruth_eventId) =', error_eventIds, 'failed', logIds.size, 'messages')

    GA = float(accurate_events) / series_groundtruth.size
    FGA = float(accurate_templates) / len(series_groundtruth.value_counts())
    
    return GA, FGA


def find_corr_oracle_templates(log_message_ids, groundtruth_df):
    """ Tìm các template chuẩn (oracle templates) tương ứng với các dòng log được trình phân tích gán template.

    Args:
        log_message_ids (pd.DataFrame): DataFrame chứa danh sách các dòng log (theo LineId) mà công cụ log parser đã gán cùng một template.
        groundtruth_df (pd.DataFrame): Dữ liệu ground truth đã được cấu trúc hóa, chứa thông tin chuẩn (oracle) về các dòng log và template tương ứng.

    Returns:
        List[str]: Danh sách các oracle template duy nhất tương ứng với tập các dòng log đã được parser gán cùng một template.

    Examples:
        log_message_ids:
        +--------+
        | LineId |
        +--------+
        |    1   |
        |    3   |
        +--------+

        groundtruth_df:
        +--------+------------------------+
        | LineId |     EventTemplate      |
        +--------+------------------------+
        |    1   | Error A happened       |
        |    2   | Disk full on node <*>  |
        |    3   | Error A happened       |
        +--------+------------------------+

        ==> Output: ["Error A happened"]
    """

    corresponding_oracle_templates = groundtruth_df.merge(log_message_ids, on='LineId')
    corresponding_oracle_templates = list(corresponding_oracle_templates.EventTemplate.unique())
    return corresponding_oracle_templates


def evaluate_template_level(dataset, groundtruth, parsedresult, output_dir):
    """ Hàm đánh giá hiệu suất phân tích log dựa trên các chỉ số: GA, PA, FTA, RTA, PTA. Đây là hàm đánh giá chung nhất, tổng quát nhất, có thể sử dụng để đánh giá cho các trình phân tích log khác nhau. Đây là tính toán theo phương thức cũ.
    """
    oracle_templates = list(pd.read_csv(groundtruth)['EventTemplate'].drop_duplicates().dropna())
    identified_templates = list(pd.read_csv(parsedresult)['EventTemplate'].drop_duplicates().dropna())
    parsedresult_df = pd.read_csv(parsedresult)
    groundtruth_df = pd.read_csv(groundtruth)

    correct_parsing_templates = 0
    for identified_template in identified_templates:

        log_message_ids = parsedresult_df.loc[parsedresult_df['EventTemplate'] == identified_template, 'LineId']
        log_message_ids = pd.DataFrame(log_message_ids)

        corr_oracle_templates = find_corr_oracle_templates(log_message_ids, groundtruth_df)

        if set(corr_oracle_templates) == {identified_template}:
            correct_parsing_templates += 1
            
    PTA = correct_parsing_templates / len(identified_templates)
    RTA = correct_parsing_templates / len(oracle_templates)
    FTA = 0.0
    if PTA != 0 or RTA != 0:
        FTA = 2 * (PTA * RTA) / (PTA + RTA)
        
    return len(identified_templates), len(oracle_templates), FTA, PTA, RTA


# ================================================================================


def is_abstract(x, y):
    if y is np.nan:
        return False

    m = re.match(get_pattern_from_template(x), y)
    if m:
        return True
    else:
        return False


def compute_template_level_accuracy(num_oracle_template, comparison_results_df):
    """
    """
    count_total = float(len(comparison_results_df))
    precision = len(comparison_results_df[comparison_results_df.type == 'SM']) / count_total  # PTA
    recall = len(comparison_results_df[comparison_results_df.type == 'SM']) / float(num_oracle_template)  # RTA
    over_generalized = len(comparison_results_df[comparison_results_df.type == 'OG']) / count_total
    under_generalized = len(comparison_results_df[comparison_results_df.type == 'UG']) / count_total
    mixed = len(comparison_results_df[comparison_results_df.type == 'MX']) / count_total
    f1_measure = 0.0
    if precision != 0 or recall != 0:
        f1_measure = 2 * (precision * recall) / (precision + recall)
    return f1_measure, precision, recall, over_generalized, under_generalized, mixed



def evaluate_template_level_all(dataset, groundtruth, parsedresult, output_dir):
    """ Thực hiện đánh giá chi tiết cấp độ mẫu (template-level evaluation) giữa các mẫu được công cụ sinh ra 
    và các mẫu gốc từ dữ liệu groundtruth.
    Hàm phân loại từng template sinh ra từ công cụ thành các loại:
    - SM (SaMe): Mẫu giống hệt với mẫu groundtruth tương ứng.
    - OG (OverGeneralized): Mẫu công cụ quá trừu tượng so với mẫu gốc.
    - UG (UnderGeneralized): Mẫu công cụ cụ thể hơn mẫu gốc.
    - MX (MiXture): Mẫu không hoàn toàn trùng, cũng không nằm hoàn toàn trong 2 nhóm trên.

    Kết quả đánh giá bao gồm:
    - Số mẫu được công cụ sinh ra và số mẫu groundtruth.
    - F1-measure, Precision (PTA), Recall (RTA) của mẫu công cụ.
    - Tỷ lệ template bị phân loại là OG, UG, MX.

    Args:
        dataset (str): Tên bộ dữ liệu đang đánh giá.
        groundtruth (str): Đường dẫn đến file CSV chứa log đã được gán nhãn (groundtruth).
        parsedresult (str): Đường dẫn đến file CSV kết quả parser sinh ra.
        output_dir (str): Thư mục lưu kết quả phân tích template.

    Returns:
        Tuple[int, int, float, float, float, float, float, float]:
            Số mẫu công cụ sinh ra, số mẫu groundtruth,
            F1, PTA, RTA, OG, UG, MX.
    """

    # Đọc các mẫu gốc (groundtruth) từ file, loại bỏ mẫu rỗng và trùng lặp.
    oracle_templates = list(pd.read_csv(groundtruth)['EventTemplate'].drop_duplicates().dropna())
    identified_templates = list(pd.read_csv(parsedresult)['EventTemplate'].drop_duplicates().dropna())
    parsedresult_df = pd.read_csv(parsedresult)
    groundtruth_df = pd.read_csv(groundtruth)

    comparison_results = []
    for identified_template in identified_templates: # Lặp qua từng template công cụ sinh ra
        identified_template_type = None

        # Get the log_message_ids corresponding to the identified template from the tool-generated structured file
        log_message_ids = parsedresult_df.loc[parsedresult_df['EventTemplate'] == identified_template, 'LineId'] # Lấy danh sách LineId (vị trí dòng log) tương ứng với template 
        log_message_ids = pd.DataFrame(log_message_ids)
        num_messages = len(log_message_ids)

        # Xác định corr_oracle_templates bằng cách sử dụng log_message_ids và oracle_structured_file
        corr_oracle_templates = find_corr_oracle_templates(log_message_ids, groundtruth_df)

        # Check SM (SaMe)
        if set(corr_oracle_templates) == {identified_template}:
            identified_template_type = 'SM'

        # incorrect template analysis
        if identified_template_type is None:

            # determine the template type
            template_types = set()
            for corr_oracle_template in corr_oracle_templates:
                if is_abstract(identified_template, corr_oracle_template):
                    template_types.add('OG')
                elif is_abstract(corr_oracle_template, identified_template):
                    template_types.add('UG')
                else:
                    template_types.add('MX')

            if len(template_types) == 1:  # if the set is singleton
                identified_template_type = template_types.pop()
            else:
                identified_template_type = 'MX'

        # save the results for the current identified template
        comparison_results.append([identified_template, identified_template_type, corr_oracle_templates, num_messages])

    comparison_results_df = pd.DataFrame(comparison_results,
                                         columns=['identified_template', 'type', 'corr_oracle_templates', 'num_messages'])
    comparison_results_df.to_csv(os.path.join(output_dir, dataset + '_template_analysis_results.csv'), index=False)
    (F1_measure, PTA, RTA, OG, UG, MX) = compute_template_level_accuracy(len(oracle_templates), comparison_results_df)
    print('F1: {:.4f}, PTA: {:.4f}, RTA: {:.4f}, OG: {:.4f}, UG: {:.4f}, MX: {:.4f}'.format(F1_measure, PTA, RTA, OG, UG, MX))
    return len(identified_templates), len(oracle_templates), F1_measure, PTA, RTA, OG, UG, MX

In [ ]:
# File evaluator.py trong LogGzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.special import comb
from sklearn.metrics import accuracy_score
import regex as re
import sys


def post_process_tokens(tokens, punc):
    """
    Phương thức xử lý danh sách token để loại bỏ các ký tự không cần thiết, sau đó chuẩn hóa lại các token.
    Cụ thể, phương thức duyệt qua từng token, nếu chứa <*>, thay thế toàn bộ bằng <*>. Sau đó, loại bỏ dấu câu được xác định trong punc trừ một số ký tự đặc biệt ['=', '|', '(', ')']. Cuối cùng, trả về danh sách token đã xử lý.
    Args:
        tokens (list): Danh sách các token đã được tách ra từ chuỗi đầu vào.
        punc (str): Chuỗi chứa các ký tự được sử dụng để loại bỏ.
    
    Returns:
        list: Danh sách các token đã được xử lý và chuẩn hóa.
    """
    excluded_str = ['=', '|', '(', ')']         # Các ký tự đặc biệt không loại bỏ.
    for i in range(len(tokens)):
        if tokens[i].find("<*>") != -1:
            tokens[i] = "<*>"                   # Ex: blk_<*> --> <*>
        else:
            # Loại bỏ các ký tự không cần thiết trong token theo danh sách punc.
            # Default: punc = "!\"#$%&'()+,-/:;=?@.[\]^_`{|}~"
            new_str = ""
            for s in tokens[i]:
                if (s not in punc and s != ' ') or s in excluded_str:
                    new_str += s
            tokens[i] = new_str
    return tokens

def message_split(message):
    """
    Chia một chuỗi đầu vào thành danh sách các token dựa trên khoảng trắng và các dấu câu đặc biệt. 
    Cụ thể, (1) Xác định các ký tự phân tách (dấu câu, khoảng trắng); (2) Sử dụng biểu thức chính quy để tách chuỗi; (3) Loại bỏ các token rỗng hoặc chỉ chứa khoảng trắng; (4)Tiền xử lý token để loại bỏ ký tự không mong muốn; (5) Xử lý trường hợp có nhiều `<*>` liên tiếp.
    Args:
        message (str): Chuỗi đầu vào cần tách.

    Returns:
        list: Danh sách các token sau khi tách.
    
    Example:
        >>> message_split("Hello, world! How are you?")
        ['Hello', ',', 'world', '!', 'How', 'are', 'you', '?']
    """
    punc = "!\"#$%&'()+,-/:;=?@.[\]^_`{|}~"                     # Các ký tự được sử dụng để tách chuỗi.
    splitters = "\s\\" + "\\".join(punc)
    splitter_regex = re.compile("([{}]+)".format(splitters))    # Tạo regex để tách chuỗi, "([{}]+)" sẽ tìm các ký tự trong splitters và giữ chúng lại trong kết quả tách.
    tokens = re.split(splitter_regex, message)                  # Tách chuỗi: "Hello,,, world.. How are you?" --> ["Hello", ",,,", "world", "..", "How", "are", "you", "?"]
    tokens = list(filter(lambda x: x != "", tokens))            # Loại bỏ các token rỗng.
    tokens = post_process_tokens(tokens, punc)                  # Xử lý hậu kỳ
    tokens = [token.strip() for token in tokens if token != "" and token != ' ']        # Loại bỏ các token rỗng và khoảng trắng.
    tokens = [token for idx, token in enumerate(tokens) if not (token == "<*>" and idx > 0 and tokens[idx - 1] == "<*>")] # Loại bỏ các token "<*>" liên tiếp.
    return tokens

def calculate_similarity(template1, template2):
    """
    Phương thức đo lường mức độ giống nhau giữa hai chuỗi văn bản (template1 và template2) bằng cách sử dụng Chỉ số Jaccard.
    Chỉ số Jaccard là tỷ lệ giữa số lượng phần tử chung của hai tập hợp và tổng số phần tử của cả hai tập hợp.
    
    Args:
        template1 (str): Chuỗi văn bản đầu tiên.
        template2 (str): Chuỗi văn bản thứ hai. 
        
    Returns:
        float: Chỉ số Jaccard giữa hai chuỗi văn bản.    
    """
    template1 = message_split(template1)
    template2 = message_split(template2)
    intersection = len(set(template1).intersection(set(template2))) # Tính số lượng phần tử chung giữa hai tập hợp.
    union = (len(template1) + len(template2)) - intersection        # Tính tổng số phần tử của cả hai tập hợp.
    return intersection / union

def evaluate_template_level(dataset, df_groundtruth, df_parsedresult, filter_templates=None):
    """
    Phương thức đánh giá chất lượng của một hệ thống trích xuất template (EventTemplate) bằng cách so sánh kết quả phân tích (df_parsedresult) với dữ liệu gốc (df_groundtruth).
    
    Args:
        dataset (str): Tên của tập dữ liệu.
        df_groundtruth (pd.DataFrame): DataFrame chứa các template gốc.
        df_parsedresult (pd.DataFrame): DataFrame chứa các template đã được phân tích.
        filter_templates (list, optional): Danh sách các template cần lọc. Mặc định là None.
    
    Returns:
        tuple: (t1, t2, FTA, PTA, RTA), trong đó:
            - t1 (int): Số lượng template được nhận diện.
            - t2 (int): Số lượng template thực tế.
            - FTA (float): F1-score của template trích xuất.
            - PTA (float): Precision (độ chính xác) của template trích xuất.
            - RTA (float): Recall (độ phủ) của template trích xuất.
    """
    correct_parsing_templates = 0
    if filter_templates is not None:
        filter_identify_templates = set()        # Lưu trữ tập hợp các template được lọc (nếu có).
    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index
    
    # Loại bỏ các dòng có giá trị NaN trong cột EventTemplate của df_groundtruth.
    df_groundtruth = df_groundtruth.loc[null_logids]
    df_parsedresult = df_parsedresult.loc[null_logids]
    
    # Tạo các Series từ cột EventTemplate của df_groundtruth và df_parsedresult, đếm số lần xuất hiện của từng template thực tế.
    series_groundtruth = df_groundtruth['EventTemplate']
    series_parsedlog = df_parsedresult['EventTemplate']
    series_groundtruth_valuecounts = series_groundtruth.value_counts()

    # Gộp dữ liệu từ df_groundtruth và df_parsedresult, nhóm theo parsedlog.
    df_combined = pd.concat([series_groundtruth, series_parsedlog], axis=1, keys=['groundtruth', 'parsedlog'])
    grouped_df = df_combined.groupby('parsedlog')

    for identified_template, group in tqdm(grouped_df):         # tqdm() thực hiện hiển thị tiến trình của vòng lặp.
        corr_oracle_templates = set(list(group['groundtruth']))
        if filter_templates is not None and len(corr_oracle_templates.intersection(set(filter_templates))) > 0:
            filter_identify_templates.add(identified_template)

        if corr_oracle_templates == {identified_template}:
            if (filter_templates is None) or (identified_template in filter_templates):
                correct_parsing_templates += 1

    if filter_templates is not None:
        PTA = correct_parsing_templates / len(filter_identify_templates)
        RTA = correct_parsing_templates / len(filter_templates)
    else:
        PTA = correct_parsing_templates / len(grouped_df)
        RTA = correct_parsing_templates / len(series_groundtruth_valuecounts)
    FTA = 0.0
    if PTA != 0 or RTA != 0:
        FTA = 2 * (PTA * RTA) / (PTA + RTA)
    print('PTA: {:.4f}, RTA: {:.4f} FTA: {:.4f}'.format(PTA, RTA, FTA))
    t1 = len(grouped_df) if filter_templates is None else len(filter_identify_templates)
    t2 = len(series_groundtruth_valuecounts) if filter_templates is None else len(filter_templates)
    print("Identify : {}, Groundtruth : {}".format(t1, t2))
    return t1, t2, FTA, PTA, RTA

def correct_lstm(groundtruth, parsedresult):
    """
    Phương thức này kiểm tra xem hai chuỗi groundtruth (chuẩn) và parsedresult (kết quả đã phân tích) có giống nhau hay không, nhưng có một điều kiện đặc biệt: Nếu một token trong groundtruth chứa "<*>", thì toàn bộ token đó sẽ được thay thế bằng "<*>", sau đó mới so sánh hai danh sách token.
    
    Args:
        groundtruth (str): Chuỗi đầu vào gốc.
        parsedresult (str): Chuỗi đầu vào đã được phân tích.    
    
    Returns:
        bool: True nếu hai chuỗi giống nhau sau khi xử lý, False nếu không.
    """
    tokens1 = groundtruth.split(' ')
    tokens2 = parsedresult.split(' ')
    tokens1 = ["<*>" if "<*>" in token else token for token in tokens1]
    return tokens1 == tokens2

def calculate_parsing_accuracy(groundtruth_df, parsedresult_df, filter_templates=None):
    """
    Phương thức tính độ chính xác của quá trình phân tích cú pháp (Parsing Accuracy - PA) dựa trên số lượng message được phân tích đúng so với tổng số message.
    
    Args:
        groundtruth_df (pd.DataFrame): DataFrame chứa các template gốc.
        parsedresult_df (pd.DataFrame): DataFrame chứa các template đã được phân tích.
        filter_templates (list, optional): Danh sách các template cần lọc. Mặc định là None.
        
    Returns:
        float: Độ chính xác của quá trình phân tích cú pháp (PA).
    """
    if filter_templates is not None:
        groundtruth_df = groundtruth_df[groundtruth_df['EventTemplate'].isin(filter_templates)]
        parsedresult_df = parsedresult_df.loc[groundtruth_df.index]
    correctly_parsed_messages = parsedresult_df[['EventTemplate']].eq(groundtruth_df[['EventTemplate']]).values.sum()
    total_messages = len(parsedresult_df[['Content']])
    PA = float(correctly_parsed_messages) / total_messages
    print('Parsing_Accuracy (PA): {:.4f}'.format(PA))
    return PA

def calculate_parsing_accuracy_lstm(groundtruth_df, parsedresult_df, filter_templates=None):
    """
    Phương thức `calculate_parsing_accuracy_lstm` được sử dụng để tính độ chính xác của việc phân tích cú pháp (Parsing Accuracy - PA) giữa dữ liệu thực tế (groundtruth_df) và dữ liệu kết quả được phân tích (parsedresult_df). Phương thức này đặc biệt sử dụng trong bối cảnh mô hình LSTM để phân tích template (Event Templates).
    
    Args:
        groundtruth_df (pd.DataFrame): DataFrame chứa các template gốc.
        parsedresult_df (pd.DataFrame): DataFrame chứa các template đã được phân tích.
        filter_templates (list, optional): Danh sách các template cần lọc. Mặc định là None.
        
    Returns:
        float: Độ chính xác của quá trình phân tích cú pháp (PA).
    """
    # parsedresult_df = pd.read_csv(parsedresult)
    # groundtruth_df = pd.read_csv(groundtruth)
    if filter_templates is not None:
        groundtruth_df = groundtruth_df[groundtruth_df['EventTemplate'].isin(filter_templates)]
        parsedresult_df = parsedresult_df.loc[groundtruth_df.index]
    # correctly_parsed_messages = parsedresult_df[['EventTemplate']].eq(groundtruth_df[['EventTemplate']]).values.sum()
    groundtruth_templates = list(groundtruth_df['EventTemplate'])
    parsedresult_templates = list(parsedresult_df['EventTemplate'])
    correctly_parsed_messages = 0
    for i in range(len(groundtruth_templates)):
        if correct_lstm(groundtruth_templates[i], parsedresult_templates[i]):
            correctly_parsed_messages += 1

    PA = float(correctly_parsed_messages) / len(groundtruth_templates)

    # similarities = []
    # for index in range(len(groundtruth_df)):
    #     similarities.append(calculate_similarity(groundtruth_df['EventTemplate'][index], parsedresult_df['EventTemplate'][index]))
    # SA = sum(similarities) / len(similarities)
    # print('Parsing_Accuracy (PA): {:.4f}, Similarity_Accuracy (SA): {:.4f}'.format(PA, SA))
    print('Parsing_Accuracy (PA): {:.4f}'.format(PA))
    return PA

def evaluate(groundtruth, parsedresult):
    df_groundtruth = pd.read_csv(groundtruth)
    df_parsedlog = pd.read_csv(parsedresult)
    
    # Remove invalid groundtruth event Ids
    non_empty_log_ids = df_groundtruth[~df_groundtruth["EventTemplate"].isnull()].index
    df_groundtruth = df_groundtruth.loc[non_empty_log_ids]
    df_parsedlog = df_parsedlog.loc[non_empty_log_ids]

    GA, FGA = get_accuracy(df_groundtruth["EventTemplate"], df_parsedlog["EventTemplate"])

    accuracy_exact_string_matching = accuracy_score(
        np.array(df_groundtruth.EventTemplate.values, dtype='str'),
        np.array(df_parsedlog.EventTemplate.values, dtype='str')
    )
    # PA = calculate_parsing_accuracy_lstm(df_groundtruth, df_parsedlog)


    _, _, FTA, PTA, RTA = evaluate_template_level(None, df_groundtruth, df_parsedlog)

    print(
        "Grouping_Accuracy (GA): {:.4f},  FGA: {:.4f}, FTA: {:.4f}, PTA: {:.4f}, RTA: {:.4f}".format(
            GA, FGA, FTA, PTA, RTA
        )
    )
    return GA, FGA, FTA, PTA, RTA

def get_accuracy(series_groundtruth, series_parsedlog, filter_templates=None):
    series_groundtruth_valuecounts = series_groundtruth.value_counts()
    series_parsedlog_valuecounts = series_parsedlog.value_counts()
    df_combined = pd.concat([series_groundtruth, series_parsedlog], axis=1, keys=['groundtruth', 'parsedlog'])
    grouped_df = df_combined.groupby('groundtruth')
    accurate_events = 0 # determine how many lines are correctly parsed
    accurate_templates = 0
    if filter_templates is not None:
        filter_identify_templates = set()
    for ground_truthId, group in tqdm(grouped_df):
        series_parsedlog_logId_valuecounts = group['parsedlog'].value_counts()
        if filter_templates is not None and ground_truthId in filter_templates:
            for parsed_eventId in series_parsedlog_logId_valuecounts.index:
                filter_identify_templates.add(parsed_eventId)
        if series_parsedlog_logId_valuecounts.size == 1:
            parsed_eventId = series_parsedlog_logId_valuecounts.index[0]
            if len(group) == series_parsedlog[series_parsedlog == parsed_eventId].size:
                if (filter_templates is None) or (ground_truthId in filter_templates):
                    accurate_events += len(group)
                    accurate_templates += 1
    if filter_templates is not None:
        GA = float(accurate_events) / len(series_groundtruth[series_groundtruth.isin(filter_templates)])
        PGA = float(accurate_templates) / len(filter_identify_templates)
        RGA = float(accurate_templates) / len(filter_templates)
    else:
        GA = float(accurate_events) / len(series_groundtruth)
        PGA = float(accurate_templates) / len(series_parsedlog_valuecounts)
        RGA = float(accurate_templates) / len(series_groundtruth_valuecounts)
    FGA = 0.0
    if PGA != 0 or RGA != 0:
        FGA = 2 * (PGA * RGA) / (PGA + RGA)
    return GA, FGA

In [ ]:
# =========================================================================
# Copyright (C) 2016-2023 LOGPAI (https://github.com/logpai).
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =========================================================================

import sys
sys.path.append("../../")
from logparser.MoLFI import LogParser
from logparser.utils import evaluator
import os
import pandas as pd


input_dir = "../../data/loghub_2k/"  # The input directory of log file
output_dir = "MoLFI_result/"  # The output directory of parsing results

benchmark_settings = {
    "HDFS": {
        "log_file": "HDFS/HDFS_2k.log",
        "log_format": "<Date> <Time> <Pid> <Level> <Component>: <Content>",
        "regex": [r"blk_-?\d+", r"(\d+\.){3}\d+(:\d+)?"],
    },
    "Hadoop": {
        "log_file": "Hadoop/Hadoop_2k.log",
        "log_format": "<Date> <Time> <Level> \[<Process>\] <Component>: <Content>",
        "regex": [r"(\d+\.){3}\d+"],
    },
    "Spark": {
        "log_file": "Spark/Spark_2k.log",
        "log_format": "<Date> <Time> <Level> <Component>: <Content>",
        "regex": [r"(\d+\.){3}\d+", r"\b[KGTM]?B\b", r"([\w-]+\.){2,}[\w-]+"],
    },
    "Zookeeper": {
        "log_file": "Zookeeper/Zookeeper_2k.log",
        "log_format": "<Date> <Time> - <Level>  \[<Node>:<Component>@<Id>\] - <Content>",
        "regex": [r"(/|)(\d+\.){3}\d+(:\d+)?"],
    },
    "HPC": {
        "log_file": "HPC/HPC_2k.log",
        "log_format": "<LogId> <Node> <Component> <State> <Time> <Flag> <Content>",
        "regex": [r"=\d+"],
    },
    "Thunderbird": {
        "log_file": "Thunderbird/Thunderbird_2k.log",
        "log_format": "<Label> <Timestamp> <Date> <User> <Month> <Day> <Time> <Location> <Component>(\[<PID>\])?: <Content>",
        "regex": [r"(\d+\.){3}\d+"],
    },
    "Windows": {
        "log_file": "Windows/Windows_2k.log",
        "log_format": "<Date> <Time>, <Level>                  <Component>    <Content>",
        "regex": [r"0x.*?\s"],
    },
    "Linux": {
        "log_file": "Linux/Linux_2k.log",
        "log_format": "<Month> <Date> <Time> <Level> <Component>(\[<PID>\])?: <Content>",
        "regex": [r"(\d+\.){3}\d+", r"\d{2}:\d{2}:\d{2}"],
    },
    "Android": {
        "log_file": "Android/Android_2k.log",
        "log_format": "<Date> <Time>  <Pid>  <Tid> <Level> <Component>: <Content>",
        "regex": [
            r"(/[\w-]+)+",
            r"([\w-]+\.){2,}[\w-]+",
            r"\b(\-?\+?\d+)\b|\b0[Xx][a-fA-F\d]+\b|\b[a-fA-F\d]{4,}\b",
        ],
    },
    "HealthApp": {
        "log_file": "HealthApp/HealthApp_2k.log",
        "log_format": "<Time>\|<Component>\|<Pid>\|<Content>",
        "regex": [],
    },

    "Proxifier": {
        "log_file": "Proxifier/Proxifier_2k.log",
        "log_format": "\[<Time>\] <Program> - <Content>",
        "regex": [
            r"<\d+\ssec",
            r"([\w-]+\.)+[\w-]+(:\d+)?",
            r"\d{2}:\d{2}(:\d{2})*",
            r"[KGTM]B",
        ],
    },
    "OpenSSH": {
        "log_file": "OpenSSH/OpenSSH_2k.log",
        "log_format": "<Date> <Day> <Time> <Component> sshd\[<Pid>\]: <Content>",
        "regex": [r"(\d+\.){3}\d+", r"([\w-]+\.){2,}[\w-]+"],
    },
    "OpenStack": {
        "log_file": "OpenStack/OpenStack_2k.log",
        "log_format": "<Logrecord> <Date> <Time> <Pid> <Level> <Component> \[<ADDR>\] <Content>",
        "regex": [r"((\d+\.){3}\d+,?)+", r"/.+?\s", r"\d+"],
    },
    "Mac": {
        "log_file": "Mac/Mac_2k.log",
        "log_format": "<Month>  <Date> <Time> <User> <Component>\[<PID>\]( \(<Address>\))?: <Content>",
        "regex": [r"([\w-]+\.){2,}[\w-]+"],
    },
}

bechmark_result = []
for dataset, setting in benchmark_settings.items():
    print("\n=== Evaluation on %s ===" % dataset)
    indir = os.path.join(input_dir, os.path.dirname(setting["log_file"]))
    log_file = os.path.basename(setting["log_file"])

    parser = LogParser(
        log_format=setting["log_format"],
        indir=indir,
        outdir=output_dir,
        rex=setting["regex"],
    )
    parser.parse(log_file)

    F1_measure, accuracy = evaluator.evaluate(
        groundtruth=os.path.join(indir, log_file + "_structured.csv"),
        parsedresult=os.path.join(output_dir, log_file + "_structured.csv"),
    )
    bechmark_result.append([dataset, F1_measure, accuracy])

print("\n=== Overall evaluation results ===")
df_result = pd.DataFrame(bechmark_result, columns=["Dataset", "F1_measure", "Accuracy"])
df_result.set_index("Dataset", inplace=True)
print(df_result)
df_result.to_csv("MoLFI_bechmark_result.csv", float_format="%.6f")
